In [1]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

print(f"Training: {len(train_data)}, Validation: {len(val_data)}, Test: {len(test_data)}")

# Enhanced transforms with stronger augmentation
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.15),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode(data, n_way, k_shot, q_query):
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        samples = class_samples[cls]
        if len(samples) < k_shot + q_query:
            # Oversample if needed
            selected = random.choices(samples, k=k_shot + q_query)
        else:
            selected = random.sample(samples, k_shot + q_query)
        
        # Support samples
        for i in range(k_shot):
            episode.append({
                'path': selected[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
        
        # Query samples  
        for i in range(k_shot, k_shot + q_query):
            episode.append({
                'path': selected[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False
            })
    
    return episode, selected_classes

class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last few layers instead of freezing everything
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with dropout and batch norm
        self.projection = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )
        
        # Temperature parameter for distance scaling
        self.temperature = nn.Parameter(torch.tensor(10.0))
        
    def forward(self, x):
        features = self.encoder(x)
        return F.normalize(self.projection(features), dim=1)  # L2 normalization

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# Model and training setup
model = ImprovedProtoNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=3000, pct_start=0.3
)

# Training loop
n_way, k_shot, q_query = 4, 1, 15
best_val_acc = 0
patience = 0
max_patience = 300

print("Starting improved training...")

for episode in tqdm(range(3000)):
    model.train()
    episode_data, _ = create_episode(train_data, n_way, k_shot, q_query)
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    # Forward pass
    support_embeddings = model(support_images)
    query_embeddings = model(query_images)
    
    # Compute prototypes with better aggregation
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    # Cosine similarity with temperature scaling
    logits = torch.mm(query_embeddings, prototypes.t()) * model.temperature
    loss = F.cross_entropy(logits, query_labels)
    
    # Add prototype regularization
    proto_reg = 0.01 * torch.mean(torch.norm(prototypes, dim=1))
    loss += proto_reg
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            for _ in range(20):  # More validation episodes
                val_episode_data, _ = create_episode(val_data, n_way, k_shot, q_query)
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_logits = torch.mm(val_query_embeddings, val_prototypes.t()) * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            
            print(f"Episode {episode} — Loss: {loss.item():.4f} — Train: {train_acc:.3f} — Val: {val_acc_avg:.3f}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
model.load_state_dict(torch.load('improved_protonet_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda
Training: 4, Validation: 9113, Test: 20872
Starting improved training...


  0%|                                                                                         | 0/3000 [00:00<?, ?it/s]

Episode 0 — Loss: 0.9211 — Train: 0.600 — Val: 0.553


  0%|                                                                              | 1/3000 [00:15<12:47:11, 15.35s/it]

✅ New best: 0.553


...

Episode 2900 — Loss: 0.0100 — Train: 1.000 — Val: 0.646


 98%|██████████████████████████████████████████████████████████████████████████▊ | 2951/3000 [1:08:57<04:35,  5.63s/it]

Episode 2950 — Loss: 0.0100 — Train: 1.000 — Val: 0.623


100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [1:09:52<00:00,  1.40s/it]


Training complete! Best validation accuracy: 0.678
Best model loaded for evaluation.


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import json
import os
import numpy as np
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Create idx_to_label mapping (missing from original)
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

# Missing prepare_batch function
def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        try:
            img = Image.open(sample['path']).convert('RGB')
            images.append(transform(img))
            labels.append(sample['episode_label'])
        except Exception as e:
            print(f"Error loading image {sample['path']}: {e}")
            continue
    
    if not images:
        return torch.empty(0), torch.empty(0, dtype=torch.long)
    
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# Missing create_fixed_support_set function
def create_fixed_support_set(data, n_way, k_shot):
    """Create a fixed support set for evaluation"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            selected = available[:k_shot]  # Take first k_shot samples
        else:
            # Repeat samples if not enough available
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def calculate_visual_dissimilarity(query_embedding, support_embeddings, support_labels, predicted_class_idx, support_paths):
    """Calculate visual dissimilarity between query and support images"""
    # Get support samples for the predicted class
    pred_class_indices = torch.where(support_labels == predicted_class_idx)[0]
    
    if len(pred_class_indices) == 0:
        return {
            'avg_dissimilarity': 1.0,
            'min_dissimilarity': 1.0,
            'max_dissimilarity': 1.0,
            'closest_support_image': None,
            'most_dissimilar_support_image': None
        }
    
    # Calculate dissimilarity (1 - cosine similarity) to each support image of predicted class
    pred_class_embeddings = support_embeddings[pred_class_indices]
    similarities = F.cosine_similarity(query_embedding.unsqueeze(0), pred_class_embeddings, dim=1)
    dissimilarities = 1 - similarities
    
    # Find indices of most and least similar support images
    min_dissim_idx = dissimilarities.argmin().item()
    max_dissim_idx = dissimilarities.argmax().item()
    
    # Get the actual support image paths
    closest_support_idx = pred_class_indices[min_dissim_idx].item()
    most_dissimilar_idx = pred_class_indices[max_dissim_idx].item()
    
    return {
        'avg_dissimilarity': dissimilarities.mean().item(),
        'min_dissimilarity': dissimilarities.min().item(),
        'max_dissimilarity': dissimilarities.max().item(),
        'closest_support_image': support_paths[closest_support_idx],
        'most_dissimilar_support_image': support_paths[most_dissimilar_idx],
        'dissimilarity_std': dissimilarities.std().item() if len(dissimilarities) > 1 else 0.0
    }

def calculate_advanced_metrics(query_embedding, prototypes, support_embeddings, support_labels, predicted_class_idx, support_paths):
    """Enhanced metrics calculation with better confidence estimation and visual dissimilarity"""
    # Cosine similarity with temperature (matching training)
    similarities = F.cosine_similarity(query_embedding.unsqueeze(0), prototypes, dim=1)
    confidences = F.softmax(similarities * 10.0, dim=0)  # Use same temperature as training
    
    # Entropy-based uncertainty
    entropy = -torch.sum(confidences * torch.log(confidences + 1e-8))
    
    # Distance to prototype
    pred_prototype = prototypes[predicted_class_idx]
    dist_to_prototype = 1 - F.cosine_similarity(query_embedding, pred_prototype, dim=0)
    
    # Margin (difference between top 2 predictions)
    sorted_conf, _ = torch.sort(confidences, descending=True)
    margin = (sorted_conf[0] - sorted_conf[1]).item() if len(sorted_conf) > 1 else 1.0
    
    # Visual dissimilarity to support images
    visual_dissim = calculate_visual_dissimilarity(
        query_embedding, support_embeddings, support_labels, predicted_class_idx, support_paths
    )
    
    return {
        'confidence': confidences[predicted_class_idx].item(),
        'entropy': entropy.item(),
        'distance_to_prototype': dist_to_prototype.item(),
        'margin': margin,
        'max_confidence': confidences.max().item(),
        **visual_dissim
    }

def test_with_tta(model, test_data, prototypes, transform_eval, device, n_tta=5):
    """Test with Test Time Augmentation for better accuracy"""
    model.eval()
    predictions = []
    
    # Create TTA transforms
    tta_transforms = [
        transform_eval,  # Original
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    ]
    
    with torch.no_grad():
        for sample in tqdm(test_data, desc="TTA Testing"):
            try:
                img = Image.open(sample['path']).convert('RGB')
                
                # Collect predictions from multiple augmentations
                tta_logits = []
                for transform in tta_transforms[:min(n_tta, len(tta_transforms))]:
                    img_tensor = transform(img).unsqueeze(0).to(device)
                    query_embedding = model(img_tensor)
                    logits = torch.mm(query_embedding, prototypes.t()) * model.temperature
                    tta_logits.append(logits)
                
                # Average logits
                avg_logits = torch.stack(tta_logits).mean(0)
                pred_idx = avg_logits.argmax().item()
                confidence = F.softmax(avg_logits, dim=1).max().item()
                
                predictions.append({
                    'predicted': pred_idx,
                    'true': label_to_idx[sample['label']],
                    'confidence': confidence,
                    'path': sample['path']
                })
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    accuracy = sum(1 for p in predictions if p['predicted'] == p['true']) / len(predictions) if predictions else 0.0
    return accuracy, predictions

def ensemble_with_diversity(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, n_ensembles=5):
    """Enhanced ensemble with diversity-based support set selection"""
    print(f"🎯 Running Diverse Ensemble (n={n_ensembles})")
    
    ensemble_predictions = []
    ensemble_accuracies = []
    
    for i in range(n_ensembles):
        # Create diverse support sets
        support_set = create_diverse_support_set(train_data, k_shot=3, diversity_factor=i)
        
        model.eval()
        support_images, support_labels = prepare_batch(support_set, transform_eval)
        
        with torch.no_grad():
            support_embeddings = model(support_images)
            
            # Compute prototypes with L2 normalization
            prototypes = []
            for cls_idx in range(len(all_labels)):
                cls_indices = torch.where(support_labels == cls_idx)[0]
                if len(cls_indices) > 0:
                    proto = support_embeddings[cls_indices].mean(0)
                    prototypes.append(F.normalize(proto, dim=0))
                else:
                    prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
            prototypes = torch.stack(prototypes)
            
            # Test with TTA
            accuracy, predictions = test_with_tta(model, test_data, prototypes, transform_eval, device, n_tta=3)
            
        ensemble_accuracies.append(accuracy)
        ensemble_predictions.append(predictions)
        print(f"  Ensemble {i+1}: {accuracy:.4f}")
    
    # Majority voting ensemble
    final_predictions = []
    for i in range(len(test_data)):
        votes = [pred[i]['predicted'] for pred in ensemble_predictions if i < len(pred)]
        confidence_scores = [pred[i]['confidence'] for pred in ensemble_predictions if i < len(pred)]
        
        if not votes:
            continue
        
        # Weighted voting based on confidence
        vote_counts = {}
        for vote, conf in zip(votes, confidence_scores):
            vote_counts[vote] = vote_counts.get(vote, 0) + conf
        
        final_pred = max(vote_counts.items(), key=lambda x: x[1])[0]
        true_label = ensemble_predictions[0][i]['true'] if i < len(ensemble_predictions[0]) else 0
        
        final_predictions.append({
            'predicted': final_pred,
            'true': true_label,
            'ensemble_votes': votes,
            'avg_confidence': np.mean(confidence_scores),
            'vote_agreement': votes.count(final_pred) / len(votes)
        })
    
    # Calculate metrics
    y_true = [p['true'] for p in final_predictions]
    y_pred = [p['predicted'] for p in final_predictions]
    
    final_accuracy = accuracy_score(y_true, y_pred) if final_predictions else 0.0
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0) if final_predictions else 0.0
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0) if final_predictions else 0.0
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0) if final_predictions else 0.0
    
    print(f"\n📊 Enhanced Ensemble Results:")
    print(f"  Final Accuracy: {final_accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Std Dev: {np.std(ensemble_accuracies):.4f}")
    
    return {
        'final_accuracy': final_accuracy,
        'individual_accuracies': ensemble_accuracies,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': final_predictions,
        'std_dev': np.std(ensemble_accuracies)
    }

def create_diverse_support_set(data, k_shot=3, diversity_factor=0):
    """Create diverse support sets for better ensemble performance"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    random.seed(diversity_factor * 42)  # Different seed for diversity
    
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            # Sample with diversity (skip some samples based on diversity_factor)
            start_idx = (diversity_factor * 2) % max(1, len(available) - k_shot)
            selected = available[start_idx:start_idx + k_shot]
            if len(selected) < k_shot:
                selected.extend(random.sample(available, k_shot - len(selected)))
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def comprehensive_prediction_analysis(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                                    transform_eval, device):
    """Comprehensive prediction analysis for all test images with top 1000 per class"""
    print("🎯 Comprehensive Prediction Analysis for All Test Images...")
    
    support_set = create_fixed_support_set(train_data, n_way=4, k_shot=1)
    model.eval()
    support_images, support_labels = prepare_batch(support_set, transform_eval)
    
    # Store support image paths for dissimilarity calculation
    support_paths = [item['path'] for item in support_set]
    
    with torch.no_grad():
        support_embeddings = model(support_images)
        
        # Compute normalized prototypes
        prototypes = []
        for cls_idx in range(len(all_labels)):
            cls_indices = torch.where(support_labels == cls_idx)[0]
            if len(cls_indices) > 0:
                proto = support_embeddings[cls_indices].mean(0)
                prototypes.append(F.normalize(proto, dim=0))
            else:
                prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
        prototypes = torch.stack(prototypes)
        
        all_predictions = []
        print(f"Processing {len(test_data)} test images...")
        
        for sample in tqdm(test_data, desc="Analyzing all test images"):
            try:
                img = Image.open(sample['path']).convert('RGB')
                img_tensor = transform_eval(img).unsqueeze(0).to(device)
                
                query_embedding = F.normalize(model(img_tensor).squeeze(), dim=0)
                
                # Get prediction
                similarities = F.cosine_similarity(query_embedding.unsqueeze(0), prototypes, dim=1)
                pred_idx = similarities.argmax().item()
                
                # Calculate advanced metrics including visual dissimilarity
                metrics = calculate_advanced_metrics(
                    query_embedding, prototypes, support_embeddings, support_labels, pred_idx, support_paths
                )
                
                all_predictions.append({
                    'image_path': sample['path'],
                    'true_label': sample['label'],
                    'predicted_label': idx_to_label[pred_idx],
                    'is_correct': pred_idx == label_to_idx[sample['label']],
                    'predicted_class_idx': pred_idx,
                    **metrics
                })
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    overall_accuracy = sum(1 for p in all_predictions if p['is_correct']) / len(all_predictions) if all_predictions else 0.0
    print(f"🎯 Overall Accuracy: {overall_accuracy:.4f}")
    
    # Group predictions by predicted class and get top 1000 most dissimilar and high confidence for each class
    class_predictions = {label: [] for label in all_labels}
    
    for pred in all_predictions:
        predicted_class = pred['predicted_label']
        class_predictions[predicted_class].append(pred)
    
    top_predictions_per_class = {}
    
    print(f"\n🏆 Extracting Top 1000 Most Dissimilar & High Confidence Predictions Per Class:")
    print("-" * 90)
    
    for class_name in all_labels:
        class_preds = class_predictions[class_name]
        if not class_preds:
            print(f"{class_name}: No predictions found")
            top_predictions_per_class[class_name] = []
            continue
        
        # Sort by dissimilarity (descending) and confidence (descending)
        # Combined score: high dissimilarity + high confidence
        for pred in class_preds:
            # Normalize dissimilarity and confidence to [0, 1] range
            dissim_score = pred.get('avg_dissimilarity', 0)
            conf_score = pred.get('confidence', 0)
            # Weighted combination: 60% dissimilarity, 40% confidence
            pred['combined_score'] = 0.6 * dissim_score + 0.4 * conf_score
        
        # Sort by combined score (higher is better for our criteria)
        class_preds.sort(key=lambda x: x['combined_score'], reverse=True)
        
        # Take top 1000 or all available if less than 1000
        top_count = min(1000, len(class_preds))
        top_predictions_per_class[class_name] = class_preds[:top_count]
        
        # Calculate statistics
        avg_dissim = np.mean([p.get('avg_dissimilarity', 0) for p in class_preds[:top_count]])
        avg_conf = np.mean([p.get('confidence', 0) for p in class_preds[:top_count]])
        accuracy_top = sum(1 for p in class_preds[:top_count] if p['is_correct']) / top_count if top_count > 0 else 0
        
        print(f"{class_name:<15}: {top_count:>4} predictions | "
              f"Avg Dissim: {avg_dissim:.4f} | Avg Conf: {avg_conf:.4f} | Accuracy: {accuracy_top:.4f}")
    
    return all_predictions, top_predictions_per_class, overall_accuracy

def analyze_predictions_by_confidence(predictions, n_bins=5):
    """Analyze prediction quality across confidence bins"""
    print(f"\n📊 Confidence-based Analysis ({n_bins} bins):")
    
    # Sort by confidence and create bins
    sorted_preds = sorted(predictions, key=lambda x: x['confidence'], reverse=True)
    bin_size = len(sorted_preds) // n_bins
    
    print(f"{'Bin':<5} {'Range':<20} {'Count':<8} {'Accuracy':<10} {'Avg Margin':<12} {'Avg Dissim':<12}")
    print("-" * 75)
    
    bin_results = []
    for i in range(n_bins):
        start_idx = i * bin_size
        end_idx = (i + 1) * bin_size if i < n_bins - 1 else len(sorted_preds)
        bin_preds = sorted_preds[start_idx:end_idx]
        
        if not bin_preds:
            continue
            
        bin_accuracy = sum(1 for p in bin_preds if p['is_correct']) / len(bin_preds)
        avg_confidence = sum(p['confidence'] for p in bin_preds) / len(bin_preds)
        avg_margin = sum(p.get('margin', 0) for p in bin_preds) / len(bin_preds)
        avg_dissim = sum(p.get('avg_dissimilarity', 0) for p in bin_preds) / len(bin_preds)
        
        conf_range = f"{bin_preds[-1]['confidence']:.3f}-{bin_preds[0]['confidence']:.3f}"
        
        print(f"{i+1:<5} {conf_range:<20} {len(bin_preds):<8} {bin_accuracy:<10.3f} {avg_margin:<12.4f} {avg_dissim:<12.4f}")
        
        bin_results.append({
            'bin': i+1,
            'accuracy': bin_accuracy,
            'avg_confidence': avg_confidence,
            'avg_dissimilarity': avg_dissim,
            'count': len(bin_preds)
        })
    
    return bin_results

def create_confusion_matrix_plot(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Create and save confusion matrix visualization"""
    try:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names)
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ Confusion matrix saved to {save_path}")
    except Exception as e:
        print(f"Error creating confusion matrix: {e}")

# Main execution
print("="*80)
print("🚀 COMPREHENSIVE PROTONET EVALUATION - ALL TEST IMAGES")
print("="*80)

# Enhanced Ensemble Analysis
ensemble_results = ensemble_with_diversity(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, n_ensembles=5
)

# Comprehensive Analysis of All Test Images
all_predictions, top_predictions_per_class, overall_accuracy = comprehensive_prediction_analysis(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, transform_eval, device
)

# Confidence-based Analysis
confidence_analysis = analyze_predictions_by_confidence(all_predictions, n_bins=5)

# Create Confusion Matrix
y_true = [label_to_idx[p['true_label']] for p in all_predictions]
y_pred = [label_to_idx[p['predicted_label']] for p in all_predictions]
create_confusion_matrix_plot(y_true, y_pred, all_labels)

# Per-class Analysis
print(f"\n📊 Per-Class Performance:")
print(f"{'Class':<12} {'Precision':<10} {'Recall':<10} {'F1':<10} {'Support':<10}")
print("-" * 55)

class_report = classification_report(y_true, y_pred, target_names=all_labels, output_dict=True, zero_division=0)
for class_name in all_labels:
    metrics = class_report[class_name]
    print(f"{class_name:<12} {metrics['precision']:<10.3f} {metrics['recall']:<10.3f} "
          f"{metrics['f1-score']:<10.3f} {int(metrics['support']):<10}")

# Top Predictions Analysis with Visual Dissimilarity for each class
print(f"\n🏆 Top 5 Most Dissimilar & High Confidence Predictions Per Class:")
print("=" * 100)
for class_name in all_labels:
    top_class_preds = top_predictions_per_class[class_name]
    if not top_class_preds:
        print(f"\n{class_name}: No predictions available")
        continue
    
    print(f"\n{class_name} (Top 5 out of {len(top_class_preds)} total):")
    print("-" * 90)
    for i, pred in enumerate(top_class_preds[:5]):
        status = "✅" if pred['is_correct'] else "❌"
        filename = os.path.basename(pred['image_path'])
        print(f"{i+1:2d}. {filename[:35]:<35} | {pred['true_label']:<12} → {pred['predicted_label']:<12}")
        print(f"     Conf: {pred['confidence']:.4f} | Dissim: {pred.get('avg_dissimilarity', 0):.4f} | "
              f"Score: {pred.get('combined_score', 0):.4f} {status}")
        if pred.get('closest_support_image'):
            closest_file = os.path.basename(pred['closest_support_image'])
            print(f"     Closest Support: {closest_file[:40]}")

# Visual Dissimilarity Analysis
print(f"\n📊 Visual Dissimilarity Analysis:")
correct_preds = [p for p in all_predictions if p['is_correct']]
incorrect_preds = [p for p in all_predictions if not p['is_correct']]

if correct_preds and incorrect_preds:
    avg_dissim_correct = np.mean([p.get('avg_dissimilarity', 0) for p in correct_preds])
    avg_dissim_incorrect = np.mean([p.get('avg_dissimilarity', 0) for p in incorrect_preds])
    print(f"  Average Dissimilarity (Correct): {avg_dissim_correct:.4f}")
    print(f"  Average Dissimilarity (Incorrect): {avg_dissim_incorrect:.4f}")
    print(f"  Dissimilarity Difference: {avg_dissim_incorrect - avg_dissim_correct:.4f}")

# Final Summary
print(f"\n📊 FINAL COMPREHENSIVE RESULTS:")
print(f"  Enhanced Ensemble: {ensemble_results['final_accuracy']:.4f} (±{ensemble_results['std_dev']:.4f})")
print(f"  Comprehensive Analysis: {overall_accuracy:.4f}")
print(f"  Total Test Images Processed: {len(all_predictions)}")
print(f"  Best Single Model: {max(ensemble_results['individual_accuracies']):.4f}")
print(f"  Weighted F1-Score: {ensemble_results['f1']:.4f}")

# Calculate total top predictions saved
total_top_predictions = sum(len(preds) for preds in top_predictions_per_class.values())
print(f"  Total Top Predictions Saved: {total_top_predictions}")
print(f"  Top Predictions Per Class: {[len(top_predictions_per_class[cls]) for cls in all_labels]}")

# Save comprehensive results with top 1000 per class
final_results = {
    'enhanced_ensemble': ensemble_results,
    'comprehensive_analysis': {
        'overall_accuracy': overall_accuracy,
        'total_predictions': len(all_predictions),
        'confidence_analysis': confidence_analysis,
        'top_predictions_per_class': top_predictions_per_class,
        'top_predictions_summary': {
            'total_saved': total_top_predictions,
            'per_class_counts': {cls: len(top_predictions_per_class[cls]) for cls in all_labels}
        },
        'visual_dissimilarity_stats': {
            'avg_dissim_correct': np.mean([p.get('avg_dissimilarity', 0) for p in correct_preds]) if correct_preds else 0,
            'avg_dissim_incorrect': np.mean([p.get('avg_dissimilarity', 0) for p in incorrect_preds]) if incorrect_preds else 0,
            'dissim_difference': (np.mean([p.get('avg_dissimilarity', 0) for p in incorrect_preds]) - 
                                 np.mean([p.get('avg_dissimilarity', 0) for p in correct_preds])) if (correct_preds and incorrect_preds) else 0
        }
    },
    'all_predictions': all_predictions,  # All predictions with detailed metrics
    'class_performance': class_report,
    'summary': {
        'best_ensemble_accuracy': ensemble_results['final_accuracy'],
        'comprehensive_accuracy': overall_accuracy,
        'ensemble_std': ensemble_results['std_dev'],
        'weighted_f1': ensemble_results['f1'],
        'total_images_processed': len(all_predictions)
    }
}
    
with open('enhanced_protonet_results.json', 'w') as f:
    json.dump(final_results, f, indent=2, default=str)

print(f"\n✅ Enhanced results saved to 'enhanced_protonet_results.json'")
print("="*80)

🚀 COMPREHENSIVE PROTONET EVALUATION - ALL TEST IMAGES
🎯 Running Diverse Ensemble (n=5)


TTA Testing: 100%|███████████████████████████████████████████████████████████████| 20872/20872 [17:51<00:00, 19.48it/s]


  Ensemble 1: 0.6610


TTA Testing: 100%|███████████████████████████████████████████████████████████████| 20872/20872 [17:03<00:00, 20.39it/s]


  Ensemble 2: 0.6605


...

  Ensemble 4: 0.6597


TTA Testing: 100%|███████████████████████████████████████████████████████████████| 20872/20872 [17:09<00:00, 20.28it/s]


  Ensemble 5: 0.6602

📊 Enhanced Ensemble Results:
  Final Accuracy: 0.6610
  Precision: 0.7325
  Recall: 0.6610
  F1-Score: 0.6237
  Std Dev: 0.0004
🎯 Comprehensive Prediction Analysis for All Test Images...
Processing 20872 test images...


Analyzing all test images: 100%|█████████████████████████████████████████████████| 20872/20872 [06:48<00:00, 51.09it/s]


🎯 Overall Accuracy: 0.6489

🏆 Extracting Top 1000 Most Dissimilar & High Confidence Predictions Per Class:
------------------------------------------------------------------------------------------
Amphibolis     : 1000 predictions | Avg Dissim: 0.4006 | Avg Conf: 0.9280 | Accuracy: 0.3570
Background     : 1000 predictions | Avg Dissim: 0.2551 | Avg Conf: 0.9471 | Accuracy: 0.6200
Halophila      : 1000 predictions | Avg Dissim: 0.3311 | Avg Conf: 0.9497 | Accuracy: 0.5960
Posidonia      : 1000 predictions | Avg Dissim: 0.2987 | Avg Conf: 0.9146 | Accuracy: 0.9170

📊 Confidence-based Analysis (5 bins):
Bin   Range                Count    Accuracy   Avg Margin   Avg Dissim  
---------------------------------------------------------------------------
1     1.000-1.000          4174     0.676      1.0000       0.0103      
2     1.000-1.000          4174     0.589      1.0000       0.0206      
3     1.000-1.000          4174     0.796      1.0000       0.0464      
4     1.000-1.000      

In [9]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# Enhanced transforms with stronger augmentation
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.15),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.6 + dissim * 0.4
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last few layers instead of freezing everything
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with dropout and batch norm
        self.projection = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )
        
        # Temperature parameter for distance scaling
        self.temperature = nn.Parameter(torch.tensor(10.0))
        
    def forward(self, x):
        features = self.encoder(x)
        return F.normalize(self.projection(features), dim=1)

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# Model and training setup
model = ImprovedProtoNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=4000, pct_start=0.3
)

# Training loop - 2-shot (1+1)
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
total_k_shot = k_shot_original + k_shot_enhanced
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 400

print(f"\n{'='*60}")
print(f"Starting 1+1 shot training (2-shot total per class)")
print(f"{'='*60}")
print(f"  - Original shots per class: {k_shot_original}")
print(f"  - Enhanced shots per class: {k_shot_enhanced}")
print(f"  - Query samples per class: {q_query}")
print(f"  - Total training episodes: 4000")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    # Forward pass
    support_embeddings = model(support_images)
    query_embeddings = model(query_images)
    
    # Compute prototypes (2 samples per class)
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    # Cosine similarity with temperature scaling
    logits = torch.mm(query_embeddings, prototypes.t()) * model.temperature
    loss = F.cross_entropy(logits, query_labels)
    
    # Add prototype regularization
    proto_reg = 0.01 * torch.mean(torch.norm(prototypes, dim=1))
    loss += proto_reg
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            for _ in range(20):
                # Create standard 1-shot validation episode
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    # 1 support sample
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    # Query samples
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_logits = torch.mm(val_query_embeddings, val_prototypes.t()) * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[0]['lr']
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | Val: {val_acc_avg:.3f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 0.9755 | Train: 0.633 | Val: 0.603 | LR: 2.00e-05


Training:   0%|                                                                    | 1/4000 [00:17<18:57:00, 17.06s/it]

✅ New best: 0.603


...

Episode 3900 | Loss: 0.0083 | Train: 1.000 | Val: 0.620 | LR: 1.51e-06


Training:  99%|█████████████████████████████████████████████████████████████████▏| 3951/4000 [1:41:26<02:39,  3.24s/it]

Episode 3950 | Loss: 0.0083 | Train: 1.000 | Val: 0.581 | LR: 3.64e-07


Training: 100%|██████████████████████████████████████████████████████████████████| 4000/4000 [1:42:16<00:00,  1.53s/it]



Training complete! Best validation accuracy: 0.745

Best model loaded for evaluation.


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import json
import os
import numpy as np
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Create idx_to_label mapping
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

def create_fixed_support_set(data, n_way, k_shot):
    """Create a fixed support set for evaluation"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            selected = available[:k_shot]
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def create_diverse_support_set(data, k_shot=2, diversity_factor=0):
    """Create diverse support sets for ensemble"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    random.seed(diversity_factor * 42)
    
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            start_idx = (diversity_factor * 2) % max(1, len(available) - k_shot)
            selected = available[start_idx:start_idx + k_shot]
            if len(selected) < k_shot:
                selected.extend(random.sample(available, k_shot - len(selected)))
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3):
    """Test with Test Time Augmentation - returns only predictions for metrics"""
    model.eval()
    predictions = []
    
    # TTA transforms
    tta_transforms = [
        transform_eval,
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    ]
    
    with torch.no_grad():
        for sample in test_data:
            try:
                img = Image.open(sample['path']).convert('RGB')
                
                tta_logits = []
                for transform in tta_transforms[:n_tta]:
                    img_tensor = transform(img).unsqueeze(0).to(device)
                    query_embedding = model(img_tensor)
                    logits = torch.mm(query_embedding, prototypes.t()) * model.temperature
                    tta_logits.append(logits)
                
                avg_logits = torch.stack(tta_logits).mean(0)
                pred_idx = avg_logits.argmax().item()
                
                predictions.append({
                    'predicted': pred_idx,
                    'true': label_to_idx[sample['label']]
                })
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    accuracy = sum(1 for p in predictions if p['predicted'] == p['true']) / len(predictions) if predictions else 0.0
    return accuracy, predictions

def ensemble_with_diversity(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, n_ensembles=5, k_shot=2):
    """Enhanced ensemble with diversity and TTA - metrics only"""
    print(f"🎯 Running Diverse Ensemble (n={n_ensembles}, k_shot={k_shot})")
    
    ensemble_predictions = []
    ensemble_accuracies = []
    
    for i in range(n_ensembles):
        support_set = create_diverse_support_set(train_data, k_shot=k_shot, diversity_factor=i)
        
        model.eval()
        support_images, support_labels = prepare_batch(support_set, transform_eval)
        
        with torch.no_grad():
            support_embeddings = model(support_images)
            
            prototypes = []
            for cls_idx in range(len(all_labels)):
                cls_indices = torch.where(support_labels == cls_idx)[0]
                if len(cls_indices) > 0:
                    proto = support_embeddings[cls_indices].mean(0)
                    prototypes.append(F.normalize(proto, dim=0))
                else:
                    prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
            prototypes = torch.stack(prototypes)
            
            accuracy, predictions = test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3)
            
        ensemble_accuracies.append(accuracy)
        ensemble_predictions.append(predictions)
        print(f"  Ensemble {i+1}: {accuracy:.4f}")
    
    # Majority voting
    final_y_true = []
    final_y_pred = []
    
    for i in range(len(test_data)):
        votes = [pred[i]['predicted'] for pred in ensemble_predictions if i < len(pred)]
        
        if not votes:
            continue
        
        # Simple majority voting
        vote_counts = {}
        for vote in votes:
            vote_counts[vote] = vote_counts.get(vote, 0) + 1
        
        final_pred = max(vote_counts.items(), key=lambda x: x[1])[0]
        true_label = ensemble_predictions[0][i]['true']
        
        final_y_true.append(true_label)
        final_y_pred.append(final_pred)
    
    # Calculate metrics
    final_accuracy = accuracy_score(final_y_true, final_y_pred)
    precision = precision_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    recall = recall_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    f1 = f1_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    
    print(f"\n📊 Enhanced Ensemble Results:")
    print(f"  Final Accuracy: {final_accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Std Dev: {np.std(ensemble_accuracies):.4f}")
    
    return {
        'y_true': final_y_true,
        'y_pred': final_y_pred,
        'final_accuracy': final_accuracy,
        'individual_accuracies': ensemble_accuracies,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'std_dev': np.std(ensemble_accuracies),
        'mean_accuracy': np.mean(ensemble_accuracies),
        'min_accuracy': np.min(ensemble_accuracies),
        'max_accuracy': np.max(ensemble_accuracies)
    }

def comprehensive_analysis(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, k_shot=2):
    """Comprehensive analysis - metrics only, no predictions saved"""
    print("🎯 Comprehensive Analysis for All Test Images...")
    
    support_set = create_fixed_support_set(train_data, n_way=len(all_labels), k_shot=k_shot)
    model.eval()
    support_images, support_labels = prepare_batch(support_set, transform_eval)
    
    with torch.no_grad():
        support_embeddings = model(support_images)
        
        prototypes = []
        for cls_idx in range(len(all_labels)):
            cls_indices = torch.where(support_labels == cls_idx)[0]
            if len(cls_indices) > 0:
                proto = support_embeddings[cls_indices].mean(0)
                prototypes.append(F.normalize(proto, dim=0))
            else:
                prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
        prototypes = torch.stack(prototypes)
        
        y_true = []
        y_pred = []
        
        print(f"Processing {len(test_data)} test images...")
        
        for sample in tqdm(test_data, desc="Analyzing test images"):
            try:
                img = Image.open(sample['path']).convert('RGB')
                img_tensor = transform_eval(img).unsqueeze(0).to(device)
                
                query_embedding = F.normalize(model(img_tensor).squeeze(), dim=0)
                
                similarities = F.cosine_similarity(query_embedding.unsqueeze(0), prototypes, dim=1)
                pred_idx = similarities.argmax().item()
                
                y_true.append(label_to_idx[sample['label']])
                y_pred.append(pred_idx)
                
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    overall_accuracy = accuracy_score(y_true, y_pred)
    print(f"🎯 Overall Accuracy: {overall_accuracy:.4f}")
    
    return y_true, y_pred, overall_accuracy

def create_confusion_matrix_plot(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Create and save confusion matrix visualization"""
    try:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names,
                    cbar_kws={'label': 'Count'})
        plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ Confusion matrix saved to {save_path}")
    except Exception as e:
        print(f"Error creating confusion matrix: {e}")

def print_classification_report(y_true, y_pred, all_labels):
    """Print detailed classification report"""
    print(f"\n📊 Per-Class Performance:")
    print(f"{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
    print("-" * 80)
    
    class_report = classification_report(y_true, y_pred, target_names=all_labels, 
                                        output_dict=True, zero_division=0)
    
    for class_name in all_labels:
        metrics = class_report[class_name]
        print(f"{class_name:<15} {metrics['precision']:<12.3f} {metrics['recall']:<12.3f} "
              f"{metrics['f1-score']:<12.3f} {int(metrics['support']):<10}")
    
    print("-" * 80)
    
    # Averages
    macro_avg = class_report['macro avg']
    weighted_avg = class_report['weighted avg']
    
    print(f"{'Macro Avg':<15} {macro_avg['precision']:<12.3f} {macro_avg['recall']:<12.3f} "
          f"{macro_avg['f1-score']:<12.3f} {int(macro_avg['support']):<10}")
    print(f"{'Weighted Avg':<15} {weighted_avg['precision']:<12.3f} {weighted_avg['recall']:<12.3f} "
          f"{weighted_avg['f1-score']:<12.3f} {int(weighted_avg['support']):<10}")
    
    return class_report

# ==================== MAIN EXECUTION ====================
print("="*80)
print("🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY")
print("="*80)

# ===== ENSEMBLE WITH DIVERSITY AND TTA =====
print("\n" + "="*80)
print("📊 ENSEMBLE EVALUATION WITH TTA")
print("="*80)

ensemble_results = ensemble_with_diversity(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, n_ensembles=5, k_shot=2
)

# Print ensemble metrics
ensemble_report = print_classification_report(ensemble_results['y_true'], 
                                             ensemble_results['y_pred'], all_labels)

# Create ensemble confusion matrix
create_confusion_matrix_plot(ensemble_results['y_true'], ensemble_results['y_pred'], 
                            all_labels, save_path='confusion_matrix_ensemble.png')

# ===== COMPREHENSIVE ANALYSIS =====
print("\n" + "="*80)
print("📊 COMPREHENSIVE SINGLE MODEL ANALYSIS")
print("="*80)

y_true_comp, y_pred_comp, overall_accuracy = comprehensive_analysis(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, k_shot=2
)

# Print comprehensive metrics
comp_report = print_classification_report(y_true_comp, y_pred_comp, all_labels)

# Create comprehensive confusion matrix
create_confusion_matrix_plot(y_true_comp, y_pred_comp, all_labels, 
                            save_path='confusion_matrix_comprehensive.png')

# ===== FINAL SUMMARY =====
print(f"\n" + "="*80)
print("📊 FINAL SUMMARY")
print("="*80)
print(f"Enhanced Ensemble: {ensemble_results['final_accuracy']:.4f} (±{ensemble_results['std_dev']:.4f})")
print(f"Comprehensive Analysis: {overall_accuracy:.4f}")
print(f"Total Test Images Processed: {len(test_data)}")
print(f"Best Single Model: {max(ensemble_results['individual_accuracies']):.4f}")
print(f"Ensemble Weighted F1-Score: {ensemble_results['f1']:.4f}")

# ===== SAVE RESULTS =====
final_results = {
    'evaluation_setup': {
        'support_shots': 2,
        'n_way': len(all_labels),
        'test_samples': len(test_data),
        'classes': all_labels,
        'ensemble_size': 5
    },
    'enhanced_ensemble': {
        'final_accuracy': ensemble_results['final_accuracy'],
        'individual_accuracies': ensemble_results['individual_accuracies'],
        'mean_accuracy': ensemble_results['mean_accuracy'],
        'std_dev': ensemble_results['std_dev'],
        'min_accuracy': ensemble_results['min_accuracy'],
        'max_accuracy': ensemble_results['max_accuracy'],
        'precision': ensemble_results['precision'],
        'recall': ensemble_results['recall'],
        'f1': ensemble_results['f1'],
        'per_class_metrics': {
            cls: {
                'precision': ensemble_report[cls]['precision'],
                'recall': ensemble_report[cls]['recall'],
                'f1_score': ensemble_report[cls]['f1-score'],
                'support': int(ensemble_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'comprehensive_analysis': {
        'overall_accuracy': overall_accuracy,
        'per_class_metrics': {
            cls: {
                'precision': comp_report[cls]['precision'],
                'recall': comp_report[cls]['recall'],
                'f1_score': comp_report[cls]['f1-score'],
                'support': int(comp_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'summary': {
        'best_ensemble_accuracy': ensemble_results['final_accuracy'],
        'comprehensive_accuracy': overall_accuracy,
        'ensemble_std': ensemble_results['std_dev'],
        'weighted_f1': ensemble_results['f1'],
        'total_images_processed': len(test_data)
    }
}

with open('evaluation_results_metrics_only.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"\n✅ Results saved to 'evaluation_results_metrics_only.json'")
print("="*80)

🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY

📊 ENSEMBLE EVALUATION WITH TTA
🎯 Running Diverse Ensemble (n=5, k_shot=2)
  Ensemble 1: 0.7378
  Ensemble 2: 0.7381
  Ensemble 3: 0.7371
  Ensemble 4: 0.7363
  Ensemble 5: 0.7380

📊 Enhanced Ensemble Results:
  Final Accuracy: 0.7384
  Precision: 0.7591
  Recall: 0.7384
  F1-Score: 0.7257
  Std Dev: 0.0007

📊 Per-Class Performance:
Class           Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Amphibolis      0.823        0.714        0.765        5375      
Background      0.794        0.426        0.554        4706      
Halophila       0.654        0.994        0.789        5669      
Posidonia       0.776        0.768        0.772        5122      
--------------------------------------------------------------------------------
Macro Avg       0.762        0.726        0.720        20872     
Weighted Avg    0.759        0.738        0.726        2

Analyzing test images: 100%|█████████████████████████████████████████████████████| 20872/20872 [06:23<00:00, 54.43it/s]


🎯 Overall Accuracy: 0.7352

📊 Per-Class Performance:
Class           Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Amphibolis      0.797        0.746        0.771        5375      
Background      0.847        0.412        0.555        4706      
Halophila       0.651        0.990        0.786        5669      
Posidonia       0.767        0.739        0.753        5122      
--------------------------------------------------------------------------------
Macro Avg       0.766        0.722        0.716        20872     
Weighted Avg    0.761        0.735        0.722        20872     
✅ Confusion matrix saved to confusion_matrix_comprehensive.png

📊 FINAL SUMMARY
Enhanced Ensemble: 0.7384 (±0.0007)
Comprehensive Analysis: 0.7352
Total Test Images Processed: 20872
Best Single Model: 0.7381
Ensemble Weighted F1-Score: 0.7257

✅ Results saved to 'evaluation_results_metrics_only.json'


In [15]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# Enhanced transforms with stronger augmentation
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.15),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.5 + dissim * 0.5
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last few layers instead of freezing everything
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with dropout and batch norm
        self.projection = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )
        
        # Temperature parameter for distance scaling
        self.temperature = nn.Parameter(torch.tensor(10.0))
        
    def forward(self, x):
        features = self.encoder(x)
        return F.normalize(self.projection(features), dim=1)

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# Model and training setup
model = ImprovedProtoNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=4000, pct_start=0.3
)

# Training loop - 2-shot (1+1)
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
total_k_shot = k_shot_original + k_shot_enhanced
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 400

print(f"\n{'='*60}")
print(f"Starting 1+1 shot training (2-shot total per class)")
print(f"{'='*60}")
print(f"  - Original shots per class: {k_shot_original}")
print(f"  - Enhanced shots per class: {k_shot_enhanced}")
print(f"  - Query samples per class: {q_query}")
print(f"  - Total training episodes: 4000")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    # Forward pass
    support_embeddings = model(support_images)
    query_embeddings = model(query_images)
    
    # Compute prototypes (2 samples per class)
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    # Cosine similarity with temperature scaling
    logits = torch.mm(query_embeddings, prototypes.t()) * model.temperature
    loss = F.cross_entropy(logits, query_labels)
    
    # Add prototype regularization
    proto_reg = 0.01 * torch.mean(torch.norm(prototypes, dim=1))
    loss += proto_reg
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            for _ in range(20):
                # Create standard 1-shot validation episode
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    # 1 support sample
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    # Query samples
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_logits = torch.mm(val_query_embeddings, val_prototypes.t()) * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[0]['lr']
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | Val: {val_acc_avg:.3f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 1.0627 | Train: 0.583 | Val: 0.617 | LR: 2.00e-05


Training:   0%|                                                                    | 1/4000 [00:14<16:29:21, 14.84s/it]

✅ New best: 0.617


...

Episode 3900 | Loss: 0.0080 | Train: 1.000 | Val: 0.626 | LR: 1.51e-06


Training:  99%|█████████████████████████████████████████████████████████████████▏| 3951/4000 [1:40:45<04:44,  5.80s/it]

Episode 3950 | Loss: 0.0078 | Train: 1.000 | Val: 0.585 | LR: 3.64e-07


Training: 100%|██████████████████████████████████████████████████████████████████| 4000/4000 [1:41:47<00:00,  1.53s/it]



Training complete! Best validation accuracy: 0.739

Best model loaded for evaluation.


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import json
import os
import numpy as np
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Create idx_to_label mapping
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

def create_fixed_support_set(data, n_way, k_shot):
    """Create a fixed support set for evaluation"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            selected = available[:k_shot]
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def create_diverse_support_set(data, k_shot=2, diversity_factor=0):
    """Create diverse support sets for ensemble"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    random.seed(diversity_factor * 42)
    
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            start_idx = (diversity_factor * 2) % max(1, len(available) - k_shot)
            selected = available[start_idx:start_idx + k_shot]
            if len(selected) < k_shot:
                selected.extend(random.sample(available, k_shot - len(selected)))
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3):
    """Test with Test Time Augmentation - returns only predictions for metrics"""
    model.eval()
    predictions = []
    
    # TTA transforms
    tta_transforms = [
        transform_eval,
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    ]
    
    with torch.no_grad():
        for sample in test_data:
            try:
                img = Image.open(sample['path']).convert('RGB')
                
                tta_logits = []
                for transform in tta_transforms[:n_tta]:
                    img_tensor = transform(img).unsqueeze(0).to(device)
                    query_embedding = model(img_tensor)
                    logits = torch.mm(query_embedding, prototypes.t()) * model.temperature
                    tta_logits.append(logits)
                
                avg_logits = torch.stack(tta_logits).mean(0)
                pred_idx = avg_logits.argmax().item()
                
                predictions.append({
                    'predicted': pred_idx,
                    'true': label_to_idx[sample['label']]
                })
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    accuracy = sum(1 for p in predictions if p['predicted'] == p['true']) / len(predictions) if predictions else 0.0
    return accuracy, predictions

def ensemble_with_diversity(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, n_ensembles=5, k_shot=2):
    """Enhanced ensemble with diversity and TTA - metrics only"""
    print(f"🎯 Running Diverse Ensemble (n={n_ensembles}, k_shot={k_shot})")
    
    ensemble_predictions = []
    ensemble_accuracies = []
    
    for i in range(n_ensembles):
        support_set = create_diverse_support_set(train_data, k_shot=k_shot, diversity_factor=i)
        
        model.eval()
        support_images, support_labels = prepare_batch(support_set, transform_eval)
        
        with torch.no_grad():
            support_embeddings = model(support_images)
            
            prototypes = []
            for cls_idx in range(len(all_labels)):
                cls_indices = torch.where(support_labels == cls_idx)[0]
                if len(cls_indices) > 0:
                    proto = support_embeddings[cls_indices].mean(0)
                    prototypes.append(F.normalize(proto, dim=0))
                else:
                    prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
            prototypes = torch.stack(prototypes)
            
            accuracy, predictions = test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3)
            
        ensemble_accuracies.append(accuracy)
        ensemble_predictions.append(predictions)
        print(f"  Ensemble {i+1}: {accuracy:.4f}")
    
    # Majority voting
    final_y_true = []
    final_y_pred = []
    
    for i in range(len(test_data)):
        votes = [pred[i]['predicted'] for pred in ensemble_predictions if i < len(pred)]
        
        if not votes:
            continue
        
        # Simple majority voting
        vote_counts = {}
        for vote in votes:
            vote_counts[vote] = vote_counts.get(vote, 0) + 1
        
        final_pred = max(vote_counts.items(), key=lambda x: x[1])[0]
        true_label = ensemble_predictions[0][i]['true']
        
        final_y_true.append(true_label)
        final_y_pred.append(final_pred)
    
    # Calculate metrics
    final_accuracy = accuracy_score(final_y_true, final_y_pred)
    precision = precision_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    recall = recall_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    f1 = f1_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    
    print(f"\n📊 Enhanced Ensemble Results:")
    print(f"  Final Accuracy: {final_accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Std Dev: {np.std(ensemble_accuracies):.4f}")
    
    return {
        'y_true': final_y_true,
        'y_pred': final_y_pred,
        'final_accuracy': final_accuracy,
        'individual_accuracies': ensemble_accuracies,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'std_dev': np.std(ensemble_accuracies),
        'mean_accuracy': np.mean(ensemble_accuracies),
        'min_accuracy': np.min(ensemble_accuracies),
        'max_accuracy': np.max(ensemble_accuracies)
    }

def comprehensive_analysis(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, k_shot=2):
    """Comprehensive analysis - metrics only, no predictions saved"""
    print("🎯 Comprehensive Analysis for All Test Images...")
    
    support_set = create_fixed_support_set(train_data, n_way=len(all_labels), k_shot=k_shot)
    model.eval()
    support_images, support_labels = prepare_batch(support_set, transform_eval)
    
    with torch.no_grad():
        support_embeddings = model(support_images)
        
        prototypes = []
        for cls_idx in range(len(all_labels)):
            cls_indices = torch.where(support_labels == cls_idx)[0]
            if len(cls_indices) > 0:
                proto = support_embeddings[cls_indices].mean(0)
                prototypes.append(F.normalize(proto, dim=0))
            else:
                prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
        prototypes = torch.stack(prototypes)
        
        y_true = []
        y_pred = []
        
        print(f"Processing {len(test_data)} test images...")
        
        for sample in tqdm(test_data, desc="Analyzing test images"):
            try:
                img = Image.open(sample['path']).convert('RGB')
                img_tensor = transform_eval(img).unsqueeze(0).to(device)
                
                query_embedding = F.normalize(model(img_tensor).squeeze(), dim=0)
                
                similarities = F.cosine_similarity(query_embedding.unsqueeze(0), prototypes, dim=1)
                pred_idx = similarities.argmax().item()
                
                y_true.append(label_to_idx[sample['label']])
                y_pred.append(pred_idx)
                
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    overall_accuracy = accuracy_score(y_true, y_pred)
    print(f"🎯 Overall Accuracy: {overall_accuracy:.4f}")
    
    return y_true, y_pred, overall_accuracy

def create_confusion_matrix_plot(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Create and save confusion matrix visualization"""
    try:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names,
                    cbar_kws={'label': 'Count'})
        plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ Confusion matrix saved to {save_path}")
    except Exception as e:
        print(f"Error creating confusion matrix: {e}")

def print_classification_report(y_true, y_pred, all_labels):
    """Print detailed classification report"""
    print(f"\n📊 Per-Class Performance:")
    print(f"{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
    print("-" * 80)
    
    class_report = classification_report(y_true, y_pred, target_names=all_labels, 
                                        output_dict=True, zero_division=0)
    
    for class_name in all_labels:
        metrics = class_report[class_name]
        print(f"{class_name:<15} {metrics['precision']:<12.3f} {metrics['recall']:<12.3f} "
              f"{metrics['f1-score']:<12.3f} {int(metrics['support']):<10}")
    
    print("-" * 80)
    
    # Averages
    macro_avg = class_report['macro avg']
    weighted_avg = class_report['weighted avg']
    
    print(f"{'Macro Avg':<15} {macro_avg['precision']:<12.3f} {macro_avg['recall']:<12.3f} "
          f"{macro_avg['f1-score']:<12.3f} {int(macro_avg['support']):<10}")
    print(f"{'Weighted Avg':<15} {weighted_avg['precision']:<12.3f} {weighted_avg['recall']:<12.3f} "
          f"{weighted_avg['f1-score']:<12.3f} {int(weighted_avg['support']):<10}")
    
    return class_report

# ==================== MAIN EXECUTION ====================
print("="*80)
print("🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY")
print("="*80)

# ===== ENSEMBLE WITH DIVERSITY AND TTA =====
print("\n" + "="*80)
print("📊 ENSEMBLE EVALUATION WITH TTA")
print("="*80)

ensemble_results = ensemble_with_diversity(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, n_ensembles=5, k_shot=2
)

# Print ensemble metrics
ensemble_report = print_classification_report(ensemble_results['y_true'], 
                                             ensemble_results['y_pred'], all_labels)

# Create ensemble confusion matrix
create_confusion_matrix_plot(ensemble_results['y_true'], ensemble_results['y_pred'], 
                            all_labels, save_path='confusion_matrix_ensemble.png')

# ===== COMPREHENSIVE ANALYSIS =====
print("\n" + "="*80)
print("📊 COMPREHENSIVE SINGLE MODEL ANALYSIS")
print("="*80)

y_true_comp, y_pred_comp, overall_accuracy = comprehensive_analysis(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, k_shot=2
)

# Print comprehensive metrics
comp_report = print_classification_report(y_true_comp, y_pred_comp, all_labels)

# Create comprehensive confusion matrix
create_confusion_matrix_plot(y_true_comp, y_pred_comp, all_labels, 
                            save_path='confusion_matrix_comprehensive.png')

# ===== FINAL SUMMARY =====
print(f"\n" + "="*80)
print("📊 FINAL SUMMARY")
print("="*80)
print(f"Enhanced Ensemble: {ensemble_results['final_accuracy']:.4f} (±{ensemble_results['std_dev']:.4f})")
print(f"Comprehensive Analysis: {overall_accuracy:.4f}")
print(f"Total Test Images Processed: {len(test_data)}")
print(f"Best Single Model: {max(ensemble_results['individual_accuracies']):.4f}")
print(f"Ensemble Weighted F1-Score: {ensemble_results['f1']:.4f}")

# ===== SAVE RESULTS =====
final_results = {
    'evaluation_setup': {
        'support_shots': 2,
        'n_way': len(all_labels),
        'test_samples': len(test_data),
        'classes': all_labels,
        'ensemble_size': 5
    },
    'enhanced_ensemble': {
        'final_accuracy': ensemble_results['final_accuracy'],
        'individual_accuracies': ensemble_results['individual_accuracies'],
        'mean_accuracy': ensemble_results['mean_accuracy'],
        'std_dev': ensemble_results['std_dev'],
        'min_accuracy': ensemble_results['min_accuracy'],
        'max_accuracy': ensemble_results['max_accuracy'],
        'precision': ensemble_results['precision'],
        'recall': ensemble_results['recall'],
        'f1': ensemble_results['f1'],
        'per_class_metrics': {
            cls: {
                'precision': ensemble_report[cls]['precision'],
                'recall': ensemble_report[cls]['recall'],
                'f1_score': ensemble_report[cls]['f1-score'],
                'support': int(ensemble_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'comprehensive_analysis': {
        'overall_accuracy': overall_accuracy,
        'per_class_metrics': {
            cls: {
                'precision': comp_report[cls]['precision'],
                'recall': comp_report[cls]['recall'],
                'f1_score': comp_report[cls]['f1-score'],
                'support': int(comp_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'summary': {
        'best_ensemble_accuracy': ensemble_results['final_accuracy'],
        'comprehensive_accuracy': overall_accuracy,
        'ensemble_std': ensemble_results['std_dev'],
        'weighted_f1': ensemble_results['f1'],
        'total_images_processed': len(test_data)
    }
}

with open('evaluation_results_metrics_only.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"\n✅ Results saved to 'evaluation_results_metrics_only.json'")
print("="*80)

🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY

📊 ENSEMBLE EVALUATION WITH TTA
🎯 Running Diverse Ensemble (n=5, k_shot=2)
  Ensemble 1: 0.7413
  Ensemble 2: 0.7405
  Ensemble 3: 0.7409
  Ensemble 4: 0.7412
  Ensemble 5: 0.7405

📊 Enhanced Ensemble Results:
  Final Accuracy: 0.7426
  Precision: 0.7636
  Recall: 0.7426
  F1-Score: 0.7305
  Std Dev: 0.0004

📊 Per-Class Performance:
Class           Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Amphibolis      0.795        0.773        0.784        5375      
Background      0.800        0.434        0.562        4706      
Halophila       0.655        0.995        0.790        5669      
Posidonia       0.818        0.716        0.763        5122      
--------------------------------------------------------------------------------
Macro Avg       0.767        0.729        0.725        20872     
Weighted Avg    0.764        0.743        0.730        2

Analyzing test images: 100%|█████████████████████████████████████████████████████| 20872/20872 [06:35<00:00, 52.81it/s]


🎯 Overall Accuracy: 0.7267

📊 Per-Class Performance:
Class           Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Amphibolis      0.752        0.773        0.762        5375      
Background      0.795        0.432        0.560        4706      
Halophila       0.653        0.990        0.787        5669      
Posidonia       0.802        0.658        0.723        5122      
--------------------------------------------------------------------------------
Macro Avg       0.751        0.713        0.708        20872     
Weighted Avg    0.747        0.727        0.714        20872     
✅ Confusion matrix saved to confusion_matrix_comprehensive.png

📊 FINAL SUMMARY
Enhanced Ensemble: 0.7426 (±0.0004)
Comprehensive Analysis: 0.7267
Total Test Images Processed: 20872
Best Single Model: 0.7413
Ensemble Weighted F1-Score: 0.7305

✅ Results saved to 'evaluation_results_metrics_only.json'


In [17]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# Enhanced transforms with stronger augmentation
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.15),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.5 + dissim * 0.5
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last few layers instead of freezing everything
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with dropout and batch norm
        self.projection = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )
        
        # Temperature parameter for distance scaling
        self.temperature = nn.Parameter(torch.tensor(10.0))
        
    def forward(self, x):
        features = self.encoder(x)
        return F.normalize(self.projection(features), dim=1)

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# Model and training setup
model = ImprovedProtoNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=4000, pct_start=0.3
)

# Training loop - 2-shot (1+1)
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
total_k_shot = k_shot_original + k_shot_enhanced
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 400

print(f"\n{'='*60}")
print(f"Starting 1+1 shot training (2-shot total per class)")
print(f"{'='*60}")
print(f"  - Original shots per class: {k_shot_original}")
print(f"  - Enhanced shots per class: {k_shot_enhanced}")
print(f"  - Query samples per class: {q_query}")
print(f"  - Total training episodes: 4000")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    # Forward pass
    support_embeddings = model(support_images)
    query_embeddings = model(query_images)
    
    # Compute prototypes (2 samples per class)
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    # Cosine similarity with temperature scaling
    logits = torch.mm(query_embeddings, prototypes.t()) * model.temperature
    loss = F.cross_entropy(logits, query_labels)
    
    # Add prototype regularization
    proto_reg = 0.01 * torch.mean(torch.norm(prototypes, dim=1))
    loss += proto_reg
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            for _ in range(20):
                # Create standard 1-shot validation episode
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    # 1 support sample
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    # Query samples
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_logits = torch.mm(val_query_embeddings, val_prototypes.t()) * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[0]['lr']
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | Val: {val_acc_avg:.3f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 1.1598 | Train: 0.450 | Val: 0.588 | LR: 2.00e-05


Training:   0%|                                                                    | 1/4000 [00:17<19:14:11, 17.32s/it]

✅ New best: 0.588


...

Episode 3900 | Loss: 0.0076 | Train: 1.000 | Val: 0.615 | LR: 1.51e-06


Training:  99%|█████████████████████████████████████████████████████████████████▏| 3951/4000 [1:40:20<04:29,  5.51s/it]

Episode 3950 | Loss: 0.0075 | Train: 1.000 | Val: 0.608 | LR: 3.64e-07


Training: 100%|██████████████████████████████████████████████████████████████████| 4000/4000 [1:41:17<00:00,  1.52s/it]



Training complete! Best validation accuracy: 0.720

Best model loaded for evaluation.


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import json
import os
import numpy as np
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Create idx_to_label mapping
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

def create_fixed_support_set(data, n_way, k_shot):
    """Create a fixed support set for evaluation"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            selected = available[:k_shot]
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def create_diverse_support_set(data, k_shot=2, diversity_factor=0):
    """Create diverse support sets for ensemble"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    random.seed(diversity_factor * 42)
    
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            start_idx = (diversity_factor * 2) % max(1, len(available) - k_shot)
            selected = available[start_idx:start_idx + k_shot]
            if len(selected) < k_shot:
                selected.extend(random.sample(available, k_shot - len(selected)))
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3):
    """Test with Test Time Augmentation - returns only predictions for metrics"""
    model.eval()
    predictions = []
    
    # TTA transforms
    tta_transforms = [
        transform_eval,
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    ]
    
    with torch.no_grad():
        for sample in test_data:
            try:
                img = Image.open(sample['path']).convert('RGB')
                
                tta_logits = []
                for transform in tta_transforms[:n_tta]:
                    img_tensor = transform(img).unsqueeze(0).to(device)
                    query_embedding = model(img_tensor)
                    logits = torch.mm(query_embedding, prototypes.t()) * model.temperature
                    tta_logits.append(logits)
                
                avg_logits = torch.stack(tta_logits).mean(0)
                pred_idx = avg_logits.argmax().item()
                
                predictions.append({
                    'predicted': pred_idx,
                    'true': label_to_idx[sample['label']]
                })
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    accuracy = sum(1 for p in predictions if p['predicted'] == p['true']) / len(predictions) if predictions else 0.0
    return accuracy, predictions

def ensemble_with_diversity(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, n_ensembles=5, k_shot=2):
    """Enhanced ensemble with diversity and TTA - metrics only"""
    print(f"🎯 Running Diverse Ensemble (n={n_ensembles}, k_shot={k_shot})")
    
    ensemble_predictions = []
    ensemble_accuracies = []
    
    for i in range(n_ensembles):
        support_set = create_diverse_support_set(train_data, k_shot=k_shot, diversity_factor=i)
        
        model.eval()
        support_images, support_labels = prepare_batch(support_set, transform_eval)
        
        with torch.no_grad():
            support_embeddings = model(support_images)
            
            prototypes = []
            for cls_idx in range(len(all_labels)):
                cls_indices = torch.where(support_labels == cls_idx)[0]
                if len(cls_indices) > 0:
                    proto = support_embeddings[cls_indices].mean(0)
                    prototypes.append(F.normalize(proto, dim=0))
                else:
                    prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
            prototypes = torch.stack(prototypes)
            
            accuracy, predictions = test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3)
            
        ensemble_accuracies.append(accuracy)
        ensemble_predictions.append(predictions)
        print(f"  Ensemble {i+1}: {accuracy:.4f}")
    
    # Majority voting
    final_y_true = []
    final_y_pred = []
    
    for i in range(len(test_data)):
        votes = [pred[i]['predicted'] for pred in ensemble_predictions if i < len(pred)]
        
        if not votes:
            continue
        
        # Simple majority voting
        vote_counts = {}
        for vote in votes:
            vote_counts[vote] = vote_counts.get(vote, 0) + 1
        
        final_pred = max(vote_counts.items(), key=lambda x: x[1])[0]
        true_label = ensemble_predictions[0][i]['true']
        
        final_y_true.append(true_label)
        final_y_pred.append(final_pred)
    
    # Calculate metrics
    final_accuracy = accuracy_score(final_y_true, final_y_pred)
    precision = precision_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    recall = recall_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    f1 = f1_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    
    print(f"\n📊 Enhanced Ensemble Results:")
    print(f"  Final Accuracy: {final_accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Std Dev: {np.std(ensemble_accuracies):.4f}")
    
    return {
        'y_true': final_y_true,
        'y_pred': final_y_pred,
        'final_accuracy': final_accuracy,
        'individual_accuracies': ensemble_accuracies,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'std_dev': np.std(ensemble_accuracies),
        'mean_accuracy': np.mean(ensemble_accuracies),
        'min_accuracy': np.min(ensemble_accuracies),
        'max_accuracy': np.max(ensemble_accuracies)
    }

def comprehensive_analysis(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, k_shot=2):
    """Comprehensive analysis - metrics only, no predictions saved"""
    print("🎯 Comprehensive Analysis for All Test Images...")
    
    support_set = create_fixed_support_set(train_data, n_way=len(all_labels), k_shot=k_shot)
    model.eval()
    support_images, support_labels = prepare_batch(support_set, transform_eval)
    
    with torch.no_grad():
        support_embeddings = model(support_images)
        
        prototypes = []
        for cls_idx in range(len(all_labels)):
            cls_indices = torch.where(support_labels == cls_idx)[0]
            if len(cls_indices) > 0:
                proto = support_embeddings[cls_indices].mean(0)
                prototypes.append(F.normalize(proto, dim=0))
            else:
                prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
        prototypes = torch.stack(prototypes)
        
        y_true = []
        y_pred = []
        
        print(f"Processing {len(test_data)} test images...")
        
        for sample in tqdm(test_data, desc="Analyzing test images"):
            try:
                img = Image.open(sample['path']).convert('RGB')
                img_tensor = transform_eval(img).unsqueeze(0).to(device)
                
                query_embedding = F.normalize(model(img_tensor).squeeze(), dim=0)
                
                similarities = F.cosine_similarity(query_embedding.unsqueeze(0), prototypes, dim=1)
                pred_idx = similarities.argmax().item()
                
                y_true.append(label_to_idx[sample['label']])
                y_pred.append(pred_idx)
                
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    overall_accuracy = accuracy_score(y_true, y_pred)
    print(f"🎯 Overall Accuracy: {overall_accuracy:.4f}")
    
    return y_true, y_pred, overall_accuracy

def create_confusion_matrix_plot(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Create and save confusion matrix visualization"""
    try:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names,
                    cbar_kws={'label': 'Count'})
        plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ Confusion matrix saved to {save_path}")
    except Exception as e:
        print(f"Error creating confusion matrix: {e}")

def print_classification_report(y_true, y_pred, all_labels):
    """Print detailed classification report"""
    print(f"\n📊 Per-Class Performance:")
    print(f"{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
    print("-" * 80)
    
    class_report = classification_report(y_true, y_pred, target_names=all_labels, 
                                        output_dict=True, zero_division=0)
    
    for class_name in all_labels:
        metrics = class_report[class_name]
        print(f"{class_name:<15} {metrics['precision']:<12.3f} {metrics['recall']:<12.3f} "
              f"{metrics['f1-score']:<12.3f} {int(metrics['support']):<10}")
    
    print("-" * 80)
    
    # Averages
    macro_avg = class_report['macro avg']
    weighted_avg = class_report['weighted avg']
    
    print(f"{'Macro Avg':<15} {macro_avg['precision']:<12.3f} {macro_avg['recall']:<12.3f} "
          f"{macro_avg['f1-score']:<12.3f} {int(macro_avg['support']):<10}")
    print(f"{'Weighted Avg':<15} {weighted_avg['precision']:<12.3f} {weighted_avg['recall']:<12.3f} "
          f"{weighted_avg['f1-score']:<12.3f} {int(weighted_avg['support']):<10}")
    
    return class_report

# ==================== MAIN EXECUTION ====================
print("="*80)
print("🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY")
print("="*80)

# ===== ENSEMBLE WITH DIVERSITY AND TTA =====
print("\n" + "="*80)
print("📊 ENSEMBLE EVALUATION WITH TTA")
print("="*80)

ensemble_results = ensemble_with_diversity(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, n_ensembles=5, k_shot=2
)

# Print ensemble metrics
ensemble_report = print_classification_report(ensemble_results['y_true'], 
                                             ensemble_results['y_pred'], all_labels)

# Create ensemble confusion matrix
create_confusion_matrix_plot(ensemble_results['y_true'], ensemble_results['y_pred'], 
                            all_labels, save_path='confusion_matrix_ensemble.png')

# ===== COMPREHENSIVE ANALYSIS =====
print("\n" + "="*80)
print("📊 COMPREHENSIVE SINGLE MODEL ANALYSIS")
print("="*80)

y_true_comp, y_pred_comp, overall_accuracy = comprehensive_analysis(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, k_shot=2
)

# Print comprehensive metrics
comp_report = print_classification_report(y_true_comp, y_pred_comp, all_labels)

# Create comprehensive confusion matrix
create_confusion_matrix_plot(y_true_comp, y_pred_comp, all_labels, 
                            save_path='confusion_matrix_comprehensive.png')

# ===== FINAL SUMMARY =====
print(f"\n" + "="*80)
print("📊 FINAL SUMMARY")
print("="*80)
print(f"Enhanced Ensemble: {ensemble_results['final_accuracy']:.4f} (±{ensemble_results['std_dev']:.4f})")
print(f"Comprehensive Analysis: {overall_accuracy:.4f}")
print(f"Total Test Images Processed: {len(test_data)}")
print(f"Best Single Model: {max(ensemble_results['individual_accuracies']):.4f}")
print(f"Ensemble Weighted F1-Score: {ensemble_results['f1']:.4f}")

# ===== SAVE RESULTS =====
final_results = {
    'evaluation_setup': {
        'support_shots': 2,
        'n_way': len(all_labels),
        'test_samples': len(test_data),
        'classes': all_labels,
        'ensemble_size': 5
    },
    'enhanced_ensemble': {
        'final_accuracy': ensemble_results['final_accuracy'],
        'individual_accuracies': ensemble_results['individual_accuracies'],
        'mean_accuracy': ensemble_results['mean_accuracy'],
        'std_dev': ensemble_results['std_dev'],
        'min_accuracy': ensemble_results['min_accuracy'],
        'max_accuracy': ensemble_results['max_accuracy'],
        'precision': ensemble_results['precision'],
        'recall': ensemble_results['recall'],
        'f1': ensemble_results['f1'],
        'per_class_metrics': {
            cls: {
                'precision': ensemble_report[cls]['precision'],
                'recall': ensemble_report[cls]['recall'],
                'f1_score': ensemble_report[cls]['f1-score'],
                'support': int(ensemble_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'comprehensive_analysis': {
        'overall_accuracy': overall_accuracy,
        'per_class_metrics': {
            cls: {
                'precision': comp_report[cls]['precision'],
                'recall': comp_report[cls]['recall'],
                'f1_score': comp_report[cls]['f1-score'],
                'support': int(comp_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'summary': {
        'best_ensemble_accuracy': ensemble_results['final_accuracy'],
        'comprehensive_accuracy': overall_accuracy,
        'ensemble_std': ensemble_results['std_dev'],
        'weighted_f1': ensemble_results['f1'],
        'total_images_processed': len(test_data)
    }
}

with open('evaluation_results_metrics_only.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"\n✅ Results saved to 'evaluation_results_metrics_only.json'")
print("="*80)

🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY

📊 ENSEMBLE EVALUATION WITH TTA
🎯 Running Diverse Ensemble (n=5, k_shot=2)
  Ensemble 1: 0.7412
  Ensemble 2: 0.7419
  Ensemble 3: 0.7425
  Ensemble 4: 0.7420
  Ensemble 5: 0.7428

📊 Enhanced Ensemble Results:
  Final Accuracy: 0.7430
  Precision: 0.7626
  Recall: 0.7430
  F1-Score: 0.7310
  Std Dev: 0.0005

📊 Per-Class Performance:
Class           Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Amphibolis      0.828        0.736        0.779        5375      
Background      0.779        0.431        0.555        4706      
Halophila       0.653        0.995        0.788        5669      
Posidonia       0.800        0.758        0.778        5122      
--------------------------------------------------------------------------------
Macro Avg       0.765        0.730        0.725        20872     
Weighted Avg    0.763        0.743        0.731        2

Analyzing test images: 100%|█████████████████████████████████████████████████████| 20872/20872 [06:27<00:00, 53.81it/s]


🎯 Overall Accuracy: 0.7371

📊 Per-Class Performance:
Class           Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Amphibolis      0.796        0.749        0.771        5375      
Background      0.782        0.438        0.562        4706      
Halophila       0.655        0.991        0.788        5669      
Posidonia       0.801        0.719        0.758        5122      
--------------------------------------------------------------------------------
Macro Avg       0.758        0.724        0.720        20872     
Weighted Avg    0.755        0.737        0.725        20872     
✅ Confusion matrix saved to confusion_matrix_comprehensive.png

📊 FINAL SUMMARY
Enhanced Ensemble: 0.7430 (±0.0005)
Comprehensive Analysis: 0.7371
Total Test Images Processed: 20872
Best Single Model: 0.7428
Ensemble Weighted F1-Score: 0.7310

✅ Results saved to 'evaluation_results_metrics_only.json'


In [20]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# Enhanced transforms with stronger augmentation
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.15),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.4 + dissim * 0.6
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last few layers instead of freezing everything
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with dropout and batch norm
        self.projection = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )
        
        # Temperature parameter for distance scaling
        self.temperature = nn.Parameter(torch.tensor(10.0))
        
    def forward(self, x):
        features = self.encoder(x)
        return F.normalize(self.projection(features), dim=1)

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# Model and training setup
model = ImprovedProtoNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=4000, pct_start=0.3
)

# Training loop - 2-shot (1+1)
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
total_k_shot = k_shot_original + k_shot_enhanced
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 400

print(f"\n{'='*60}")
print(f"Starting 1+1 shot training (2-shot total per class)")
print(f"{'='*60}")
print(f"  - Original shots per class: {k_shot_original}")
print(f"  - Enhanced shots per class: {k_shot_enhanced}")
print(f"  - Query samples per class: {q_query}")
print(f"  - Total training episodes: 4000")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    # Forward pass
    support_embeddings = model(support_images)
    query_embeddings = model(query_images)
    
    # Compute prototypes (2 samples per class)
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    # Cosine similarity with temperature scaling
    logits = torch.mm(query_embeddings, prototypes.t()) * model.temperature
    loss = F.cross_entropy(logits, query_labels)
    
    # Add prototype regularization
    proto_reg = 0.01 * torch.mean(torch.norm(prototypes, dim=1))
    loss += proto_reg
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            for _ in range(20):
                # Create standard 1-shot validation episode
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    # 1 support sample
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    # Query samples
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_logits = torch.mm(val_query_embeddings, val_prototypes.t()) * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[0]['lr']
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | Val: {val_acc_avg:.3f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 1.1230 | Train: 0.500 | Val: 0.575 | LR: 2.00e-05


Training:   0%|                                                                    | 1/4000 [00:16<18:42:13, 16.84s/it]

✅ New best: 0.575


...

Episode 3900 | Loss: 0.0070 | Train: 1.000 | Val: 0.603 | LR: 1.51e-06


Training:  99%|█████████████████████████████████████████████████████████████████▏| 3951/4000 [1:37:58<04:27,  5.46s/it]

Episode 3950 | Loss: 0.0072 | Train: 1.000 | Val: 0.588 | LR: 3.64e-07


Training: 100%|██████████████████████████████████████████████████████████████████| 4000/4000 [1:38:56<00:00,  1.48s/it]



Training complete! Best validation accuracy: 0.703

Best model loaded for evaluation.


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import json
import os
import numpy as np
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Create idx_to_label mapping
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

def create_fixed_support_set(data, n_way, k_shot):
    """Create a fixed support set for evaluation"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            selected = available[:k_shot]
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def create_diverse_support_set(data, k_shot=2, diversity_factor=0):
    """Create diverse support sets for ensemble"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    random.seed(diversity_factor * 42)
    
    for cls_idx, cls in enumerate(all_labels):
        available = class_samples[cls]
        if len(available) >= k_shot:
            start_idx = (diversity_factor * 2) % max(1, len(available) - k_shot)
            selected = available[start_idx:start_idx + k_shot]
            if len(selected) < k_shot:
                selected.extend(random.sample(available, k_shot - len(selected)))
        else:
            selected = available * (k_shot // len(available) + 1)
            selected = selected[:k_shot]
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3):
    """Test with Test Time Augmentation - returns only predictions for metrics"""
    model.eval()
    predictions = []
    
    # TTA transforms
    tta_transforms = [
        transform_eval,
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    ]
    
    with torch.no_grad():
        for sample in test_data:
            try:
                img = Image.open(sample['path']).convert('RGB')
                
                tta_logits = []
                for transform in tta_transforms[:n_tta]:
                    img_tensor = transform(img).unsqueeze(0).to(device)
                    query_embedding = model(img_tensor)
                    logits = torch.mm(query_embedding, prototypes.t()) * model.temperature
                    tta_logits.append(logits)
                
                avg_logits = torch.stack(tta_logits).mean(0)
                pred_idx = avg_logits.argmax().item()
                
                predictions.append({
                    'predicted': pred_idx,
                    'true': label_to_idx[sample['label']]
                })
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    accuracy = sum(1 for p in predictions if p['predicted'] == p['true']) / len(predictions) if predictions else 0.0
    return accuracy, predictions

def ensemble_with_diversity(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, n_ensembles=5, k_shot=2):
    """Enhanced ensemble with diversity and TTA - metrics only"""
    print(f"🎯 Running Diverse Ensemble (n={n_ensembles}, k_shot={k_shot})")
    
    ensemble_predictions = []
    ensemble_accuracies = []
    
    for i in range(n_ensembles):
        support_set = create_diverse_support_set(train_data, k_shot=k_shot, diversity_factor=i)
        
        model.eval()
        support_images, support_labels = prepare_batch(support_set, transform_eval)
        
        with torch.no_grad():
            support_embeddings = model(support_images)
            
            prototypes = []
            for cls_idx in range(len(all_labels)):
                cls_indices = torch.where(support_labels == cls_idx)[0]
                if len(cls_indices) > 0:
                    proto = support_embeddings[cls_indices].mean(0)
                    prototypes.append(F.normalize(proto, dim=0))
                else:
                    prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
            prototypes = torch.stack(prototypes)
            
            accuracy, predictions = test_with_tta(model, test_data, prototypes, transform_eval, device, label_to_idx, n_tta=3)
            
        ensemble_accuracies.append(accuracy)
        ensemble_predictions.append(predictions)
        print(f"  Ensemble {i+1}: {accuracy:.4f}")
    
    # Majority voting
    final_y_true = []
    final_y_pred = []
    
    for i in range(len(test_data)):
        votes = [pred[i]['predicted'] for pred in ensemble_predictions if i < len(pred)]
        
        if not votes:
            continue
        
        # Simple majority voting
        vote_counts = {}
        for vote in votes:
            vote_counts[vote] = vote_counts.get(vote, 0) + 1
        
        final_pred = max(vote_counts.items(), key=lambda x: x[1])[0]
        true_label = ensemble_predictions[0][i]['true']
        
        final_y_true.append(true_label)
        final_y_pred.append(final_pred)
    
    # Calculate metrics
    final_accuracy = accuracy_score(final_y_true, final_y_pred)
    precision = precision_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    recall = recall_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    f1 = f1_score(final_y_true, final_y_pred, average='weighted', zero_division=0)
    
    print(f"\n📊 Enhanced Ensemble Results:")
    print(f"  Final Accuracy: {final_accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Std Dev: {np.std(ensemble_accuracies):.4f}")
    
    return {
        'y_true': final_y_true,
        'y_pred': final_y_pred,
        'final_accuracy': final_accuracy,
        'individual_accuracies': ensemble_accuracies,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'std_dev': np.std(ensemble_accuracies),
        'mean_accuracy': np.mean(ensemble_accuracies),
        'min_accuracy': np.min(ensemble_accuracies),
        'max_accuracy': np.max(ensemble_accuracies)
    }

def comprehensive_analysis(model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
                          transform_eval, device, k_shot=2):
    """Comprehensive analysis - metrics only, no predictions saved"""
    print("🎯 Comprehensive Analysis for All Test Images...")
    
    support_set = create_fixed_support_set(train_data, n_way=len(all_labels), k_shot=k_shot)
    model.eval()
    support_images, support_labels = prepare_batch(support_set, transform_eval)
    
    with torch.no_grad():
        support_embeddings = model(support_images)
        
        prototypes = []
        for cls_idx in range(len(all_labels)):
            cls_indices = torch.where(support_labels == cls_idx)[0]
            if len(cls_indices) > 0:
                proto = support_embeddings[cls_indices].mean(0)
                prototypes.append(F.normalize(proto, dim=0))
            else:
                prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
        prototypes = torch.stack(prototypes)
        
        y_true = []
        y_pred = []
        
        print(f"Processing {len(test_data)} test images...")
        
        for sample in tqdm(test_data, desc="Analyzing test images"):
            try:
                img = Image.open(sample['path']).convert('RGB')
                img_tensor = transform_eval(img).unsqueeze(0).to(device)
                
                query_embedding = F.normalize(model(img_tensor).squeeze(), dim=0)
                
                similarities = F.cosine_similarity(query_embedding.unsqueeze(0), prototypes, dim=1)
                pred_idx = similarities.argmax().item()
                
                y_true.append(label_to_idx[sample['label']])
                y_pred.append(pred_idx)
                
            except Exception as e:
                print(f"Error processing {sample['path']}: {e}")
                continue
    
    overall_accuracy = accuracy_score(y_true, y_pred)
    print(f"🎯 Overall Accuracy: {overall_accuracy:.4f}")
    
    return y_true, y_pred, overall_accuracy

def create_confusion_matrix_plot(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Create and save confusion matrix visualization"""
    try:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names,
                    cbar_kws={'label': 'Count'})
        plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ Confusion matrix saved to {save_path}")
    except Exception as e:
        print(f"Error creating confusion matrix: {e}")

def print_classification_report(y_true, y_pred, all_labels):
    """Print detailed classification report"""
    print(f"\n📊 Per-Class Performance:")
    print(f"{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
    print("-" * 80)
    
    class_report = classification_report(y_true, y_pred, target_names=all_labels, 
                                        output_dict=True, zero_division=0)
    
    for class_name in all_labels:
        metrics = class_report[class_name]
        print(f"{class_name:<15} {metrics['precision']:<12.3f} {metrics['recall']:<12.3f} "
              f"{metrics['f1-score']:<12.3f} {int(metrics['support']):<10}")
    
    print("-" * 80)
    
    # Averages
    macro_avg = class_report['macro avg']
    weighted_avg = class_report['weighted avg']
    
    print(f"{'Macro Avg':<15} {macro_avg['precision']:<12.3f} {macro_avg['recall']:<12.3f} "
          f"{macro_avg['f1-score']:<12.3f} {int(macro_avg['support']):<10}")
    print(f"{'Weighted Avg':<15} {weighted_avg['precision']:<12.3f} {weighted_avg['recall']:<12.3f} "
          f"{weighted_avg['f1-score']:<12.3f} {int(weighted_avg['support']):<10}")
    
    return class_report

# ==================== MAIN EXECUTION ====================
print("="*80)
print("🚀 COMPREHENSIVE PROTONET EVALUATION - METRICS ONLY")
print("="*80)

# ===== ENSEMBLE WITH DIVERSITY AND TTA =====
print("\n" + "="*80)
print("📊 ENSEMBLE EVALUATION WITH TTA")
print("="*80)

ensemble_results = ensemble_with_diversity(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, n_ensembles=5, k_shot=2
)

# Print ensemble metrics
ensemble_report = print_classification_report(ensemble_results['y_true'], 
                                             ensemble_results['y_pred'], all_labels)

# Create ensemble confusion matrix
create_confusion_matrix_plot(ensemble_results['y_true'], ensemble_results['y_pred'], 
                            all_labels, save_path='confusion_matrix_ensemble.png')

# ===== COMPREHENSIVE ANALYSIS =====
print("\n" + "="*80)
print("📊 COMPREHENSIVE SINGLE MODEL ANALYSIS")
print("="*80)

y_true_comp, y_pred_comp, overall_accuracy = comprehensive_analysis(
    model, test_data, train_data, all_labels, label_to_idx, idx_to_label, 
    transform_eval, device, k_shot=2
)

# Print comprehensive metrics
comp_report = print_classification_report(y_true_comp, y_pred_comp, all_labels)

# Create comprehensive confusion matrix
create_confusion_matrix_plot(y_true_comp, y_pred_comp, all_labels, 
                            save_path='confusion_matrix_comprehensive.png')

# ===== FINAL SUMMARY =====
print(f"\n" + "="*80)
print("📊 FINAL SUMMARY")
print("="*80)
print(f"Enhanced Ensemble: {ensemble_results['final_accuracy']:.4f} (±{ensemble_results['std_dev']:.4f})")
print(f"Comprehensive Analysis: {overall_accuracy:.4f}")
print(f"Total Test Images Processed: {len(test_data)}")
print(f"Best Single Model: {max(ensemble_results['individual_accuracies']):.4f}")
print(f"Ensemble Weighted F1-Score: {ensemble_results['f1']:.4f}")

# ===== SAVE RESULTS =====
final_results = {
    'evaluation_setup': {
        'support_shots': 2,
        'n_way': len(all_labels),
        'test_samples': len(test_data),
        'classes': all_labels,
        'ensemble_size': 5
    },
    'enhanced_ensemble': {
        'final_accuracy': ensemble_results['final_accuracy'],
        'individual_accuracies': ensemble_results['individual_accuracies'],
        'mean_accuracy': ensemble_results['mean_accuracy'],
        'std_dev': ensemble_results['std_dev'],
        'min_accuracy': ensemble_results['min_accuracy'],
        'max_accuracy': ensemble_results['max_accuracy'],
        'precision': ensemble_results['precision'],
        'recall': ensemble_results['recall'],
        'f1': ensemble_results['f1'],
        'per_class_metrics': {
            cls: {
                'precision': ensemble_report[cls]['precision'],
                'recall': ensemble_report[cls]['recall'],
                'f1_score': ensemble_report[cls]['f1-score'],
                'support': int(ensemble_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'comprehensive_analysis': {
        'overall_accuracy': overall_accuracy,
        'per_class_metrics': {
            cls: {
                'precision': comp_report[cls]['precision'],
                'recall': comp_report[cls]['recall'],
                'f1_score': comp_report[cls]['f1-score'],
                'support': int(comp_report[cls]['support'])
            }
            for cls in all_labels
        }
    },
    'summary': {
        'best_ensemble_accuracy': ensemble_results['final_accuracy'],
        'comprehensive_accuracy': overall_accuracy,
        'ensemble_std': ensemble_results['std_dev'],
        'weighted_f1': ensemble_results['f1'],
        'total_images_processed': len(test_data)
    }
}

with open('evaluation_results_metrics_only.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"\n✅ Results saved to 'evaluation_results_metrics_only.json'")
print("="*80)

In [25]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# FIXED: More conservative augmentation to prevent overfitting
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),  # Very mild crop
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.15, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.4 + dissim * 0.6
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

# FIXED: Simpler, more stable architecture
class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune only the last layer to prevent overfitting
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Simpler projection to prevent overfitting
        self.projection = nn.Sequential(
            nn.Linear(feature_dim, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),  # Higher dropout
            nn.Linear(256, 128)
        )
        
        # Fixed temperature (no learning to prevent instability)
        self.temperature = 10.0
        
    def forward(self, x):
        features = self.encoder(x)
        projected = self.projection(features)
        return F.normalize(projected, dim=1)

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

# FIXED: More conservative optimizer settings
model = ImprovedProtoNet().to(device)

# Single optimizer with conservative settings
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

# Simple step LR scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)

# Training loop - 2-shot (1+1)
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
total_k_shot = k_shot_original + k_shot_enhanced
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 300

print(f"\n{'='*60}")
print(f"Starting 1+1 shot training (2-shot total per class)")
print(f"{'='*60}")
print(f"  - Original shots per class: {k_shot_original}")
print(f"  - Enhanced shots per class: {k_shot_enhanced}")
print(f"  - Query samples per class: {q_query}")
print(f"  - Total training episodes: 4000")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    # Forward pass
    support_embeddings = model(support_images)
    query_embeddings = model(query_images)
    
    # Compute prototypes (simple averaging)
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    # Compute distances (negative Euclidean distance)
    dists = torch.cdist(query_embeddings, prototypes)
    logits = -dists * model.temperature
    
    # Simple cross-entropy loss
    loss = F.cross_entropy(logits, query_labels)
    
    # FIXED: Small L2 regularization on embeddings only
    l2_reg = 0.001 * torch.mean(torch.norm(query_embeddings, dim=1))
    total_loss = loss + l2_reg
    
    optimizer.zero_grad()
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            for _ in range(20):
                # Create standard 1-shot validation episode
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    # 1 support sample
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    # Query samples
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_dists = torch.cdist(val_query_embeddings, val_prototypes)
                val_logits = -val_dists * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            val_acc_std = np.std(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[0]['lr']
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | "
                  f"Val: {val_acc_avg:.3f}±{val_acc_std:.3f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 0.9663 | Train: 0.600 | Val: 0.554±0.082 | LR: 5.00e-05


Training:   0%|                                                                    | 1/4000 [00:18<20:29:45, 18.45s/it]

✅ New best: 0.554


...

Episode 3600 | Loss: 0.0000 | Train: 1.000 | Val: 0.648±0.114 | LR: 6.25e-06


Training:  91%|████████████████████████████████████████████████████████████▏     | 3651/4000 [1:29:52<34:59,  6.01s/it]

Episode 3650 | Loss: 0.0000 | Train: 1.000 | Val: 0.649±0.097 | LR: 6.25e-06


Training:  92%|█████████████████████████████████████████████████████████████     | 3700/4000 [1:31:00<07:22,  1.48s/it]


KeyboardInterrupt: 

In [26]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# ENHANCED: Multiple augmentation views with test-time augmentation compatibility
transform_train_strong = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.2, hue=0.08),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_train_weak = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.95, 1.0)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.4 + dissim * 0.6
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

# ENHANCED: Better architecture with residual connections and multi-scale features
class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last 2 layers for better feature extraction
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with residual-like connections
        self.projection1 = nn.Sequential(
            nn.Linear(feature_dim, 384),
            nn.BatchNorm1d(384),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        self.projection2 = nn.Sequential(
            nn.Linear(384, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        self.projection3 = nn.Linear(256, 128)
        
        # Learnable temperature with constraints
        self.log_temperature = nn.Parameter(torch.log(torch.tensor(10.0)))
        
    def forward(self, x, return_intermediate=False):
        features = self.encoder(x)
        
        # Multi-stage projection
        h1 = self.projection1(features)
        h2 = self.projection2(h1)
        h3 = self.projection3(h2)
        
        if return_intermediate:
            return F.normalize(h3, dim=1), F.normalize(h2, dim=1)
        return F.normalize(h3, dim=1)
    
    @property
    def temperature(self):
        # Constrain temperature to reasonable range [5, 20]
        return torch.clamp(torch.exp(self.log_temperature), 5.0, 20.0)

def prepare_batch(batch, transform, multi_view=False):
    """Prepare batch with optional multi-view augmentation"""
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    
    if multi_view and random.random() < 0.3:  # 30% chance to use second view
        images2 = []
        for sample in batch:
            img = Image.open(sample['path']).convert('RGB')
            images2.append(transform(img))
        return torch.stack(images).to(device), torch.stack(images2).to(device), torch.tensor(labels).to(device)
    
    return torch.stack(images).to(device), None, torch.tensor(labels).to(device)

# ENHANCED: Multi-prototype approach
def compute_prototypes_robust(embeddings, labels, n_way, num_augments=1):
    """Compute prototypes with optional multi-view averaging"""
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(labels == cls)[0]
        cls_embeddings = embeddings[cls_indices]
        
        # Average over all support samples for this class
        proto = cls_embeddings.mean(0)
        prototypes.append(proto)
    
    return torch.stack(prototypes)

# ENHANCED: Label smoothing for better generalization
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
    
    def forward(self, pred, target):
        n_class = pred.size(1)
        one_hot = torch.zeros_like(pred).scatter(1, target.unsqueeze(1), 1)
        one_hot = one_hot * (1 - self.smoothing) + self.smoothing / n_class
        log_prob = F.log_softmax(pred, dim=1)
        return -(one_hot * log_prob).sum(dim=1).mean()

# Model setup
model = ImprovedProtoNet().to(device)

# Differential learning rates
clip_params = [p for n, p in model.named_parameters() if 'clip_model' in n or 'encoder' in n]
other_params = [p for n, p in model.named_parameters() if not ('clip_model' in n or 'encoder' in n)]

optimizer = torch.optim.AdamW([
    {'params': clip_params, 'lr': 2e-5, 'weight_decay': 1e-4},
    {'params': other_params, 'lr': 1e-4, 'weight_decay': 1e-3}
])

# Cosine annealing with warmup
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=4000, eta_min=1e-6)

# Label smoothing loss
criterion = LabelSmoothingCrossEntropy(smoothing=0.05)

# Training configuration
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 400

print(f"\n{'='*60}")
print(f"Starting ENHANCED 1+1 shot training")
print(f"{'='*60}")
print(f"  - Architecture: Multi-stage projection with BatchNorm")
print(f"  - Loss: Label smoothing (0.05)")
print(f"  - Augmentation: Dual-strength with multi-view")
print(f"  - Temperature: Learnable with constraints [5, 20]")
print(f"  - Episodes: 4000 with cosine annealing")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    
    # Alternate between strong and weak augmentation
    use_strong_aug = episode % 2 == 0
    current_transform = transform_train_strong if use_strong_aug else transform_train_weak
    
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    # Prepare with multi-view augmentation
    support_images, support_images2, support_labels = prepare_batch(support_set, transform_eval, multi_view=False)
    query_images, query_images2, query_labels = prepare_batch(query_set, current_transform, multi_view=True)
    
    # Forward pass - support
    support_embeddings = model(support_images)
    
    # Forward pass - query (with optional second view)
    query_embeddings = model(query_images)
    if query_images2 is not None:
        query_embeddings2 = model(query_images2)
        query_embeddings = (query_embeddings + query_embeddings2) / 2  # Average views
    
    # Compute prototypes
    prototypes = compute_prototypes_robust(support_embeddings, support_labels, n_way)
    
    # Compute distances with learnable temperature
    dists = torch.cdist(query_embeddings, prototypes)
    logits = -dists * model.temperature
    
    # Label smoothing loss
    loss = criterion(logits, query_labels)
    
    # Regularization: encourage diverse prototypes
    proto_sim = torch.mm(prototypes, prototypes.t())
    proto_mask = 1 - torch.eye(n_way).to(device)
    proto_diversity = 0.02 * (proto_sim * proto_mask).sum() / (n_way * (n_way - 1))
    
    # Regularization: prevent temperature from becoming too extreme
    temp_reg = 0.001 * (model.temperature - 10.0).abs()
    
    total_loss = loss + proto_diversity + temp_reg
    
    optimizer.zero_grad()
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            
            for _ in range(25):  # More validation episodes
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, _, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, _, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = compute_prototypes_robust(val_support_embeddings, val_support_labels, n_way)
                
                val_query_embeddings = model(val_query_images)
                val_dists = torch.cdist(val_query_embeddings, val_prototypes)
                val_logits = -val_dists * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            val_acc_std = np.std(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[1]['lr']
            current_temp = model.temperature.item()
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | "
                  f"Val: {val_acc_avg:.3f}±{val_acc_std:.3f} | Temp: {current_temp:.2f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 1.1806 | Train: 0.483 | Val: 0.573±0.094 | Temp: 10.00 | LR: 1.00e-04


Training:   0%|                                                                    | 1/4000 [00:14<16:35:06, 14.93s/it]

✅ New best: 0.573


...

Episode 3900 | Loss: 0.2036 | Train: 1.000 | Val: 0.655±0.102 | Temp: 10.00 | LR: 1.15e-06


Training:  99%|█████████████████████████████████████████████████████████████████▏| 3951/4000 [1:55:57<05:48,  7.11s/it]

Episode 3950 | Loss: 0.2035 | Train: 1.000 | Val: 0.636±0.120 | Temp: 10.00 | LR: 1.04e-06


Training: 100%|██████████████████████████████████████████████████████████████████| 4000/4000 [1:57:05<00:00,  1.76s/it]



Training complete! Best validation accuracy: 0.677

Best model loaded for evaluation.


In [32]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.cuda.amp as amp
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    if isinstance(data, dict) and 'images' in data:
        return data['images']
    elif isinstance(data, list):
        return data
    else:
        raise ValueError(f"Unexpected JSON structure in {json_path}. Expected either a list or dict with 'images' key.")

def filter_by_index(data, target_indexes):
    """Filter data to only include items with specific indexes"""
    return [item for item in data if item['index'] in target_indexes]

# Load data from new paths
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Filter training data to only include indexes 1, 16, 31, 46
target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

print(f"Filtered training data: {len(train_data)} samples from indexes {target_indexes}")
print(f"Validation data: {len(val_data)} samples")
print(f"Test data: {len(test_data)} samples")

# New class labels based on your data
all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

print(f"Classes: {all_labels}")

# Advanced training transforms with augmentation
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomCrop(224, padding=20, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAutocontrast(p=0.2),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Simpler evaluation transforms
transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode(data, n_way, k_shot, q_query, allow_training_in_query=True, max_training_in_query=10):
    episode = []
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    
    for cls_idx, cls in enumerate(selected_classes):
        available_samples = class_samples[cls]
        
        if len(available_samples) < k_shot + q_query:
            support_samples = random.sample(available_samples, min(k_shot, len(available_samples)))
            
            if allow_training_in_query:
                remaining_needed = q_query
                query_samples = []
                
                non_support_samples = [s for s in available_samples if s not in support_samples]
                if non_support_samples:
                    take_from_non_support = min(remaining_needed, len(non_support_samples))
                    query_samples.extend(random.sample(non_support_samples, take_from_non_support))
                    remaining_needed -= take_from_non_support
                
                if remaining_needed > 0:
                    additional_samples = random.choices(available_samples, k=min(remaining_needed, max_training_in_query))
                    query_samples.extend(additional_samples)
            else:
                query_samples = random.sample(available_samples, q_query)
        else:
            samples = random.sample(available_samples, k_shot + q_query)
            support_samples = samples[:k_shot]
            query_samples = samples[k_shot:]
        
        for i, sample in enumerate(support_samples):
            episode.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
        
        for sample in query_samples:
            episode.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False
            })
    
    return episode, selected_classes

# ProtoNet class definition with improvements
class ProtoNet(nn.Module):
    def __init__(self):
        super(ProtoNet, self).__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Freeze base CLIP parameters
        for param in self.clip_model.parameters():
            param.requires_grad = False
            
        self.encoder = self.clip_model.visual
            
        # Improved projection head with dropout and layer norm
        self.projection = nn.Sequential(
            nn.Linear(512, 512),
            nn.LayerNorm(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 256)
        )
        
    def forward(self, x):
        with torch.no_grad():
            features = self.encoder(x)
        return self.projection(features)

def prepare_batch(batch, transform):
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    return torch.stack(images).to(device), torch.tensor(labels).to(device)

model = ProtoNet().to(device)

# Improved optimizer settings
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=1e-3,  # Higher initial learning rate
    weight_decay=1e-3,  # Stronger regularization
    betas=(0.9, 0.999)
)

# More aggressive learning rate schedule
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-3,
    total_steps=5000,
    pct_start=0.3,
    anneal_strategy='cos'
)

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)  # Add label smoothing

# Training parameters
n_way = 4
k_shot = 1
q_query = 10
best_val_acc = 0
best_model_state = None

print("\nStarting meta-training...")
for episode in tqdm(range(5000)):  # Increased to 5000 episodes
    model.train()
    episode_data, _ = create_episode(train_data, n_way, k_shot, q_query, 
                                     allow_training_in_query=True, 
                                     max_training_in_query=10)
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    support_images, support_labels = prepare_batch(support_set, transform_train)
    query_images, query_labels = prepare_batch(query_set, transform_train)
    
    support_embeddings = model(support_images)
    
    # Compute class prototypes
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(support_labels == cls)[0]
        cls_proto = support_embeddings[cls_indices].mean(0)
        prototypes.append(cls_proto)
    prototypes = torch.stack(prototypes)
    
    query_embeddings = model(query_images)
    
    # Use negative squared euclidean distance with temperature scaling
    temperature = 10.0  # Temperature parameter for better gradients
    dists = -torch.cdist(query_embeddings, prototypes)**2 / temperature
    loss = loss_fn(dists, query_labels)
    
    optimizer.zero_grad()
    loss.backward()
    
    # Gradient clipping to prevent instability
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    scheduler.step()
    
    # Validation every 100 episodes
    if episode % 100 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            val_losses = []
            
            for val_ep in range(20):  # More validation episodes for stability
                val_episode_data, _ = create_episode(val_data, n_way, k_shot, q_query)
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = []
                for cls in range(n_way):
                    cls_indices = torch.where(val_support_labels == cls)[0]
                    val_proto = val_support_embeddings[cls_indices].mean(0)
                    val_prototypes.append(val_proto)
                val_prototypes = torch.stack(val_prototypes)
                
                val_query_embeddings = model(val_query_images)
                val_dists = -torch.cdist(val_query_embeddings, val_prototypes)**2 / temperature
                val_preds = torch.argmax(val_dists, dim=1)
                
                val_loss = loss_fn(val_dists, val_query_labels)
                val_acc = (val_preds == val_query_labels).float().mean().item()
                
                val_accuracies.append(val_acc)
                val_losses.append(val_loss.item())
            
            train_acc = (torch.argmax(dists, dim=1) == query_labels).float().mean().item()
            val_acc_avg = sum(val_accuracies) / len(val_accuracies)
            val_loss_avg = sum(val_losses) / len(val_losses)
            current_lr = optimizer.param_groups[0]['lr']
            
            print(f"Episode {episode} — Loss: {loss.item():.4f} — Val Loss: {val_loss_avg:.4f} — Train Acc: {train_acc:.2%} — Val Acc: {val_acc_avg:.2%} — LR: {current_lr:.6f}")
            
            # Save best model
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                best_model_state = model.state_dict().copy()
                print(f"✅ New best model with validation accuracy: {best_val_acc:.2%}")

# Save best model
torch.save(best_model_state, 'clip_seagrass_1shot_filtered_improved.pth')
print(f"\nBest validation accuracy: {best_val_acc:.2%}")

# Load the best model for evaluation
model.load_state_dict(torch.load('clip_seagrass_1shot_filtered_improved.pth'))
print("Best model loaded for final evaluation.")

Using device: cuda
Filtered training data: 4 samples from indexes [1, 16, 31, 46]
Validation data: 9113 samples
Test data: 20872 samples
Classes: ['Amphibolis', 'Background', 'Halophila', 'Posidonia']

Starting meta-training...


  0%|                                                                              | 1/5000 [00:11<16:19:00, 11.75s/it]

Episode 0 — Loss: 1.2690 — Val Loss: 1.1547 — Train Acc: 40.00% — Val Acc: 59.63% — LR: 0.000040
✅ New best model with validation accuracy: 59.63%


  2%|█▌                                                                           | 101/5000 [01:49<5:52:50,  4.32s/it]

Episode 100 — Loss: 0.5425 — Val Loss: 1.0543 — Train Acc: 92.50% — Val Acc: 61.50% — LR: 0.000051
✅ New best model with validation accuracy: 61.50%


...

Episode 4800 — Loss: 0.3537 — Val Loss: 0.9526 — Train Acc: 100.00% — Val Acc: 67.00% — LR: 0.000008


 98%|██████████████████████████████████████████████████████████████████████████▍ | 4901/5000 [1:19:19<07:16,  4.41s/it]

Episode 4900 — Loss: 0.3516 — Val Loss: 0.9150 — Train Acc: 100.00% — Val Acc: 68.75% — LR: 0.000002


100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [1:20:43<00:00,  1.03it/s]



Best validation accuracy: 74.50%
Best model loaded for final evaluation.


In [33]:
import os
import json
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['images'] if isinstance(data, dict) and 'images' in data else data

def filter_by_index(data, target_indexes):
    return [item for item in data if item['index'] in target_indexes]

# Load and filter data
train_data_full = load_data(r"C:\Users\matin\ECU\code\newcodes\jsoncreation\trainseed123.json")
val_data = load_data(r"C:\Users\matin\ECU\Validation_Patches\validation.json")
test_data = load_data(r"C:\Users\matin\ECU\Test_Patches\Test.json")

# Load enhanced results for additional shots
enhanced_results_path = r"C:\Users\matin\ECU\code\newcodes\enhanced_protonet_results.json"
with open(enhanced_results_path, 'r') as f:
    enhanced_data_raw = json.load(f)

# Extract ALL predictions from the JSON structure
enhanced_data = []
if isinstance(enhanced_data_raw, dict):
    if 'all_predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['all_predictions']
    elif 'predictions' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['predictions']
    elif 'top_predictions_per_class' in enhanced_data_raw:
        print("Loading from 'top_predictions_per_class'...")
        for class_name, predictions in enhanced_data_raw['top_predictions_per_class'].items():
            if isinstance(predictions, list):
                enhanced_data.extend(predictions)
                print(f"  - {class_name}: {len(predictions)} predictions")
    elif 'results' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['results']
    elif 'images' in enhanced_data_raw:
        enhanced_data = enhanced_data_raw['images']
    else:
        print(f"Available keys in JSON: {list(enhanced_data_raw.keys())}")
        enhanced_data = []
elif isinstance(enhanced_data_raw, list):
    enhanced_data = enhanced_data_raw
else:
    raise ValueError(f"Unexpected JSON structure. Type: {type(enhanced_data_raw)}")

print(f"\nTotal loaded: {len(enhanced_data)} samples from enhanced results")

target_indexes = [1, 16, 31, 46]
train_data = filter_by_index(train_data_full, target_indexes)

all_labels = ['Amphibolis', 'Background', 'Halophila', 'Posidonia']
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Organize enhanced data by true label - ONLY CORRECT predictions
enhanced_by_label = {label: [] for label in all_labels}
skipped_count = 0
correct_count = 0
incorrect_count = 0

if len(enhanced_data) > 0:
    print(f"\nFirst sample keys: {list(enhanced_data[0].keys())}")
    print(f"First sample preview: {list(enhanced_data[0].items())[:3]}")

for item in enhanced_data:
    if 'true_label' not in item:
        skipped_count += 1
        continue
    
    true_label = item['true_label']
    is_correct = item.get('is_correct', False)
    
    if is_correct:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # CRITICAL: Only include correctly predicted samples
    if true_label in enhanced_by_label and is_correct:
        enhanced_by_label[true_label].append({
            'path': item.get('image_path', ''),
            'label': true_label,
            'confidence': item.get('confidence', 0.0),
            'is_correct': item['is_correct'],
            'distance_to_prototype': item.get('distance_to_prototype', 0.0),
            'max_dissimilarity': item.get('max_dissimilarity', 0.0),
            'entropy': item.get('entropy', 0.0)
        })

print(f"\nProcessing summary:")
print(f"  - Total samples processed: {len(enhanced_data)}")
print(f"  - Correct predictions: {correct_count}")
print(f"  - Incorrect predictions: {incorrect_count}")
print(f"  - Skipped (no true_label): {skipped_count}")

print(f"Training (original): {len(train_data)}")
print(f"Enhanced data available (correct predictions only):")
for label, items in enhanced_by_label.items():
    if len(items) > 0:
        avg_conf = np.mean([x['confidence'] for x in items])
        avg_dissim = np.mean([x['max_dissimilarity'] for x in items])
        print(f"  {label}: {len(items)} images (avg conf: {avg_conf:.3f}, avg dissim: {avg_dissim:.3f})")
    else:
        print(f"  {label}: {len(items)} images")
print(f"Validation: {len(val_data)}, Test: {len(test_data)}")

# ENHANCED: Multiple augmentation views with test-time augmentation compatibility
transform_train_strong = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.2, hue=0.08),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_train_weak = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.95, 1.0)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def create_episode_1plus1(data, enhanced_data_by_label, n_way, k_shot_original, k_shot_enhanced, q_query):
    """
    Create episode with 1+1 shot learning (2 shots total per class):
    - k_shot_original: shots from original training data
    - k_shot_enhanced: shots from enhanced results (CORRECT, HIGH CONFIDENCE, MOST DISSIMILAR)
    Total support set per class = 2
    """
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    selected_classes = random.sample(all_labels, n_way)
    episode = []
    
    for cls_idx, cls in enumerate(selected_classes):
        # ===== PART 1: Original training samples (1 shot) =====
        original_samples = class_samples[cls]
        if len(original_samples) < k_shot_original + q_query:
            selected_original = random.choices(original_samples, k=k_shot_original + q_query)
        else:
            selected_original = random.sample(original_samples, k_shot_original + q_query)
        
        # ===== PART 2: Enhanced samples (1 shot - STRATEGIC SELECTION) =====
        enhanced_samples = enhanced_by_label[cls]
        
        if len(enhanced_samples) >= k_shot_enhanced:
            # Strategy: Select samples with HIGH CONFIDENCE and HIGH DISSIMILARITY
            scored_samples = []
            for sample in enhanced_samples:
                conf = sample['confidence']
                dissim = sample['max_dissimilarity']
                
                # Combined score: prioritize high confidence AND high dissimilarity
                score = conf * 0.4 + dissim * 0.6
                scored_samples.append((score, sample))
            
            # Sort by score (descending) and select top k
            scored_samples.sort(key=lambda x: x[0], reverse=True)
            selected_enhanced = [item[1] for item in scored_samples[:k_shot_enhanced]]
            
        elif len(enhanced_samples) > 0:
            selected_enhanced = random.choices(enhanced_samples, k=k_shot_enhanced)
        else:
            # Fallback: use additional original samples
            if len(original_samples) >= k_shot_original + k_shot_enhanced + q_query:
                selected_enhanced = [original_samples[k_shot_original]]
            else:
                selected_enhanced = random.choices(original_samples, k=k_shot_enhanced)
        
        # ===== Add samples to episode =====
        # Add 1 original support sample
        for i in range(k_shot_original):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'original'
            })
        
        # Add 1 enhanced support sample
        for enhanced_item in selected_enhanced:
            episode.append({
                'path': enhanced_item['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True,
                'source': 'enhanced',
                'confidence': enhanced_item['confidence'],
                'dissimilarity': enhanced_item['max_dissimilarity']
            })
        
        # Query samples (from original data only)
        for i in range(k_shot_original, k_shot_original + q_query):
            episode.append({
                'path': selected_original[i]['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': False,
                'source': 'original'
            })
    
    return episode, selected_classes

# ENHANCED: Better architecture with residual connections and multi-scale features
class ImprovedProtoNet(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        import clip
        self.clip_model, _ = clip.load("ViT-B/32", device=device)
        self.clip_model = self.clip_model.float()
        
        # Fine-tune last 2 layers for better feature extraction
        for name, param in self.clip_model.named_parameters():
            if 'transformer.resblocks.11' in name or 'transformer.resblocks.10' in name or 'ln_post' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        self.encoder = self.clip_model.visual
        
        # Enhanced projection with residual-like connections
        self.projection1 = nn.Sequential(
            nn.Linear(feature_dim, 384),
            nn.BatchNorm1d(384),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        self.projection2 = nn.Sequential(
            nn.Linear(384, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        self.projection3 = nn.Linear(256, 128)
        
        # Learnable temperature with constraints
        self.log_temperature = nn.Parameter(torch.log(torch.tensor(10.0)))
        
    def forward(self, x, return_intermediate=False):
        features = self.encoder(x)
        
        # Multi-stage projection
        h1 = self.projection1(features)
        h2 = self.projection2(h1)
        h3 = self.projection3(h2)
        
        if return_intermediate:
            return F.normalize(h3, dim=1), F.normalize(h2, dim=1)
        return F.normalize(h3, dim=1)
    
    @property
    def temperature(self):
        # Constrain temperature to reasonable range [5, 20]
        return torch.clamp(torch.exp(self.log_temperature), 5.0, 20.0)

def prepare_batch(batch, transform, multi_view=False):
    """Prepare batch with optional multi-view augmentation"""
    images, labels = [], []
    for sample in batch:
        img = Image.open(sample['path']).convert('RGB')
        images.append(transform(img))
        labels.append(sample['episode_label'])
    
    if multi_view and random.random() < 0.3:  # 30% chance to use second view
        images2 = []
        for sample in batch:
            img = Image.open(sample['path']).convert('RGB')
            images2.append(transform(img))
        return torch.stack(images).to(device), torch.stack(images2).to(device), torch.tensor(labels).to(device)
    
    return torch.stack(images).to(device), None, torch.tensor(labels).to(device)

# ENHANCED: Multi-prototype approach
def compute_prototypes_robust(embeddings, labels, n_way, num_augments=1):
    """Compute prototypes with optional multi-view averaging"""
    prototypes = []
    for cls in range(n_way):
        cls_indices = torch.where(labels == cls)[0]
        cls_embeddings = embeddings[cls_indices]
        
        # Average over all support samples for this class
        proto = cls_embeddings.mean(0)
        prototypes.append(proto)
    
    return torch.stack(prototypes)

# ENHANCED: Label smoothing for better generalization
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
    
    def forward(self, pred, target):
        n_class = pred.size(1)
        one_hot = torch.zeros_like(pred).scatter(1, target.unsqueeze(1), 1)
        one_hot = one_hot * (1 - self.smoothing) + self.smoothing / n_class
        log_prob = F.log_softmax(pred, dim=1)
        return -(one_hot * log_prob).sum(dim=1).mean()

# Model setup
model = ImprovedProtoNet().to(device)

# Differential learning rates
clip_params = [p for n, p in model.named_parameters() if 'clip_model' in n or 'encoder' in n]
other_params = [p for n, p in model.named_parameters() if not ('clip_model' in n or 'encoder' in n)]

optimizer = torch.optim.AdamW([
    {'params': clip_params, 'lr': 2e-5, 'weight_decay': 1e-4},
    {'params': other_params, 'lr': 1e-4, 'weight_decay': 1e-3}
])

# Cosine annealing with warmup
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=4000, eta_min=1e-6)

# Label smoothing loss
criterion = LabelSmoothingCrossEntropy(smoothing=0.05)

# Training configuration
n_way = 4
k_shot_original = 1
k_shot_enhanced = 1
q_query = 15

best_val_acc = 0
patience = 0
max_patience = 400

print(f"\n{'='*60}")
print(f"Starting ENHANCED 1+1 shot training")
print(f"{'='*60}")
print(f"  - Architecture: Multi-stage projection with BatchNorm")
print(f"  - Loss: Label smoothing (0.05)")
print(f"  - Augmentation: Dual-strength with multi-view")
print(f"  - Temperature: Learnable with constraints [5, 20]")
print(f"  - Episodes: 4000 with cosine annealing")
print(f"{'='*60}\n")

for episode in tqdm(range(4000), desc="Training"):
    model.train()
    
    # Alternate between strong and weak augmentation
    use_strong_aug = episode % 2 == 0
    current_transform = transform_train_strong if use_strong_aug else transform_train_weak
    
    episode_data, _ = create_episode_1plus1(
        train_data, enhanced_by_label, n_way, 
        k_shot_original, k_shot_enhanced, q_query
    )
    
    support_set = [s for s in episode_data if s['is_support']]
    query_set = [q for q in episode_data if not q['is_support']]
    
    # Prepare with multi-view augmentation
    support_images, support_images2, support_labels = prepare_batch(support_set, transform_eval, multi_view=False)
    query_images, query_images2, query_labels = prepare_batch(query_set, current_transform, multi_view=True)
    
    # Forward pass - support
    support_embeddings = model(support_images)
    
    # Forward pass - query (with optional second view)
    query_embeddings = model(query_images)
    if query_images2 is not None:
        query_embeddings2 = model(query_images2)
        query_embeddings = (query_embeddings + query_embeddings2) / 2  # Average views
    
    # Compute prototypes
    prototypes = compute_prototypes_robust(support_embeddings, support_labels, n_way)
    
    # Compute distances with learnable temperature
    dists = torch.cdist(query_embeddings, prototypes)
    logits = -dists * model.temperature
    
    # Label smoothing loss
    loss = criterion(logits, query_labels)
    
    # Regularization: encourage diverse prototypes
    proto_sim = torch.mm(prototypes, prototypes.t())
    proto_mask = 1 - torch.eye(n_way).to(device)
    proto_diversity = 0.02 * (proto_sim * proto_mask).sum() / (n_way * (n_way - 1))
    
    # Regularization: prevent temperature from becoming too extreme
    temp_reg = 0.001 * (model.temperature - 10.0).abs()
    
    total_loss = loss + proto_diversity + temp_reg
    
    optimizer.zero_grad()
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    # Validation every 50 episodes
    if episode % 50 == 0:
        model.eval()
        with torch.no_grad():
            val_accuracies = []
            
            for _ in range(25):  # More validation episodes
                class_samples_val = {label: [] for label in all_labels}
                for item in val_data:
                    class_samples_val[item['label']].append(item)
                
                selected_classes_val = random.sample(all_labels, n_way)
                val_episode_data = []
                
                for cls_idx, cls in enumerate(selected_classes_val):
                    samples = class_samples_val[cls]
                    if len(samples) < 1 + q_query:
                        selected = random.choices(samples, k=1 + q_query)
                    else:
                        selected = random.sample(samples, 1 + q_query)
                    
                    val_episode_data.append({
                        'path': selected[0]['path'],
                        'label': cls,
                        'episode_label': cls_idx,
                        'is_support': True
                    })
                    
                    for i in range(1, 1 + q_query):
                        val_episode_data.append({
                            'path': selected[i]['path'],
                            'label': cls,
                            'episode_label': cls_idx,
                            'is_support': False
                        })
                
                val_support = [s for s in val_episode_data if s['is_support']]
                val_query = [q for q in val_episode_data if not q['is_support']]
                
                val_support_images, _, val_support_labels = prepare_batch(val_support, transform_eval)
                val_query_images, _, val_query_labels = prepare_batch(val_query, transform_eval)
                
                val_support_embeddings = model(val_support_images)
                val_prototypes = compute_prototypes_robust(val_support_embeddings, val_support_labels, n_way)
                
                val_query_embeddings = model(val_query_images)
                val_dists = torch.cdist(val_query_embeddings, val_prototypes)
                val_logits = -val_dists * model.temperature
                val_preds = torch.argmax(val_logits, dim=1)
                
                val_acc = (val_preds == val_query_labels).float().mean().item()
                val_accuracies.append(val_acc)
            
            val_acc_avg = np.mean(val_accuracies)
            val_acc_std = np.std(val_accuracies)
            train_acc = (torch.argmax(logits, dim=1) == query_labels).float().mean().item()
            current_lr = optimizer.param_groups[1]['lr']
            current_temp = model.temperature.item()
            
            print(f"Episode {episode:4d} | Loss: {loss.item():.4f} | Train: {train_acc:.3f} | "
                  f"Val: {val_acc_avg:.3f}±{val_acc_std:.3f} | Temp: {current_temp:.2f} | LR: {current_lr:.2e}")
            
            if val_acc_avg > best_val_acc:
                best_val_acc = val_acc_avg
                torch.save(model.state_dict(), 'improved_protonet_1plus1_best.pth')
                patience = 0
                print(f"✅ New best: {best_val_acc:.3f}")
            else:
                patience += 1
                if patience >= max_patience:
                    print("Early stopping triggered")
                    break

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.3f}")
print(f"{'='*60}\n")
model.load_state_dict(torch.load('improved_protonet_1plus1_best.pth'))
print("Best model loaded for evaluation.")

# ============================================================================
# TEST-TIME EVALUATION CODE (Compatible with the provided test code)
# ============================================================================

def create_fixed_support_set(data, n_way=4, k_shot=1):
    """Create fixed support set for testing"""
    class_samples = {label: [] for label in all_labels}
    for item in data:
        class_samples[item['label']].append(item)
    
    support_set = []
    for cls_idx, cls in enumerate(all_labels):
        samples = class_samples[cls]
        # Use first k_shot samples for consistency
        selected = samples[:k_shot] if len(samples) >= k_shot else samples
        
        for sample in selected:
            support_set.append({
                'path': sample['path'],
                'label': cls,
                'episode_label': cls_idx,
                'is_support': True
            })
    
    return support_set

def predict_all_with_confidence(model, test_data):
    """Predict all test images with confidence scores using TTA"""
    support_set = create_fixed_support_set(train_data, n_way=4, k_shot=1)
    
    model.eval()
    support_images, support_labels = prepare_batch(support_set, transform_eval)
    
    predictions = []
    
    with torch.no_grad():
        support_embeddings = model(support_images)
        
        # Compute class prototypes
        prototypes = []
        for cls_idx in range(len(all_labels)):
            cls_indices = torch.where(support_labels == cls_idx)[0]
            if len(cls_indices) > 0:
                cls_proto = support_embeddings[cls_indices].mean(0)
                prototypes.append(cls_proto)
            else:
                prototypes.append(torch.zeros(support_embeddings.shape[1]).to(device))
        
        prototypes = torch.stack(prototypes)
        
        # Test-time augmentation transforms
        tta_transforms = [
            transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(p=1.0),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ]
        
        # Get temperature value (handle both property and parameter)
        temp_value = model.temperature if isinstance(model.temperature, (int, float)) else model.temperature.item()
        
        for i, sample in enumerate(tqdm(test_data, desc="Predicting all test images")):
            img = Image.open(sample['path']).convert('RGB')
            true_label = label_to_idx[sample['label']]
            
            # Apply TTA
            embeddings_list = []
            for transform in tta_transforms:
                img_tensor = transform(img).unsqueeze(0).to(device)
                embeddings = model(img_tensor)
                embeddings_list.append(embeddings)
            
            avg_embedding = torch.mean(torch.cat(embeddings_list, dim=0), dim=0, keepdim=True)
            
            # Calculate distances with temperature scaling
            dists = torch.cdist(avg_embedding, prototypes)
            logits = -dists * temp_value
            
            # Convert to probabilities using softmax for confidence scores
            confidence_scores = torch.softmax(logits, dim=1).squeeze()
            
            # Get predicted class
            pred_idx = torch.argmax(logits, dim=1).item()
            pred_label = all_labels[pred_idx]
            max_confidence = confidence_scores[pred_idx].item()
            
            predictions.append({
                'image_index': i,
                'image_path': sample['path'],
                'true_label': sample['label'],
                'predicted_label': pred_label,
                'confidence': max_confidence,
                'is_correct': pred_idx == true_label,
                'all_confidences': {all_labels[j]: confidence_scores[j].item() for j in range(len(all_labels))}
            })
    
    # Sort by confidence (highest first)
    predictions.sort(key=lambda x: x['confidence'], reverse=True)
    
    return predictions

# Get predictions for all test images
print("\n" + "="*80)
print("PREDICTING ALL TEST IMAGES AND GETTING TOP 15 BY CONFIDENCE")
print("="*80)

all_predictions = predict_all_with_confidence(model, test_data)

# Display top 15 most confident predictions
print("\nTOP 15 MOST CONFIDENT PREDICTIONS:")
print("="*50)

for i, pred in enumerate(all_predictions[:15]):
    print(f"\nRank {i+1}:")
    print(f"Image: {os.path.basename(pred['image_path'])}")
    print(f"True Label: {pred['true_label']}")
    print(f"Predicted Label: {pred['predicted_label']}")
    print(f"Confidence: {pred['confidence']:.4f}")
    print(f"Correct: {'✅' if pred['is_correct'] else '❌'}")
    print("All class confidences:")
    for label, conf in pred['all_confidences'].items():
        print(f"  {label}: {conf:.4f}")

# Calculate overall accuracy
correct_predictions = sum(1 for pred in all_predictions if pred['is_correct'])
total_predictions = len(all_predictions)
accuracy = correct_predictions / total_predictions

print(f"\n" + "="*50)
print(f"FINAL RESULTS")
print(f"="*50)
print(f"Total test images: {total_predictions}")
print(f"Correct predictions: {correct_predictions}")
print(f"Overall accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Save results to JSON
results_data = {
    'overall_accuracy': accuracy,
    'total_images': total_predictions,
    'correct_predictions': correct_predictions,
    'top_15_predictions': all_predictions[:15],
    'all_predictions': all_predictions
}

with open('test_predictions_with_confidence_seagrass_1shot_filtered.json', 'w') as f:
    json.dump(results_data, f, indent=4)

print(f"\nResults saved to 'test_predictions_with_confidence_seagrass_1shot_filtered.json'")

# Get predictions for all test images - Per Class Analysis
print("\n" + "="*80)
print("PREDICTING ALL TEST IMAGES AND GETTING TOP 15 BY CONFIDENCE PER CLASS")
print("="*80)

# Group predictions by predicted class and get top 15 for each
predictions_by_class = {}
for pred in all_predictions:
    predicted_class = pred['predicted_label']
    if predicted_class not in predictions_by_class:
        predictions_by_class[predicted_class] = []
    predictions_by_class[predicted_class].append(pred)

# Sort each class by confidence and get top 15
top15_per_class = {}
for class_name, class_predictions in predictions_by_class.items():
    # Sort by confidence (highest first)
    class_predictions.sort(key=lambda x: x['confidence'], reverse=True)
    top15_per_class[class_name] = class_predictions[:15]

# Display top 15 for each class
print("\nTOP 15 MOST CONFIDENT PREDICTIONS PER CLASS:")
print("="*60)

for class_name, top_predictions in top15_per_class.items():
    print(f"\n🏷️  CLASS: {class_name.upper()}")
    print("-" * 40)
    
    for i, pred in enumerate(top_predictions):
        print(f"  Rank {i+1}:")
        print(f"    Image: {os.path.basename(pred['image_path'])}")
        print(f"    True Label: {pred['true_label']}")
        print(f"    Confidence: {pred['confidence']:.4f}")
        print(f"    Correct: {'✅' if pred['is_correct'] else '❌'}")
        
        # Show top 3 confidence scores for this prediction
        sorted_confidences = sorted(pred['all_confidences'].items(), 
                                  key=lambda x: x[1], reverse=True)
        print(f"    Top confidences: {', '.join([f'{label}:{conf:.3f}' for label, conf in sorted_confidences[:3]])}")
        print()

# Calculate per-class statistics
print("\n" + "="*60)
print("PER-CLASS STATISTICS")
print("="*60)

class_stats = {}
for class_name in all_labels:
    # Get all predictions for this class (both correct and incorrect)
    class_preds = [p for p in all_predictions if p['predicted_label'] == class_name]
    correct_preds = [p for p in class_preds if p['is_correct']]
    
    # Also get true instances of this class
    true_instances = [p for p in all_predictions if p['true_label'] == class_name]
    correctly_identified = [p for p in true_instances if p['is_correct']]
    
    class_stats[class_name] = {
        'predicted_as_this_class': len(class_preds),
        'correctly_predicted_as_this_class': len(correct_preds),
        'true_instances': len(true_instances),
        'correctly_identified_instances': len(correctly_identified),
        'precision': len(correct_preds) / len(class_preds) if class_preds else 0,
        'recall': len(correctly_identified) / len(true_instances) if true_instances else 0
    }

for class_name, stats in class_stats.items():
    precision = stats['precision']
    recall = stats['recall']
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n📊 {class_name.upper()}:")
    print(f"   Predicted as this class: {stats['predicted_as_this_class']}")
    print(f"   Correctly predicted: {stats['correctly_predicted_as_this_class']}")
    print(f"   True instances: {stats['true_instances']}")
    print(f"   Correctly identified: {stats['correctly_identified_instances']}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall: {recall:.4f}")
    print(f"   F1-Score: {f1:.4f}")

# Calculate overall accuracy
print(f"\n" + "="*50)
print(f"OVERALL RESULTS")
print(f"="*50)
print(f"Total test images: {total_predictions}")
print(f"Correct predictions: {correct_predictions}")
print(f"Overall accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Save enhanced results to JSON
results_data = {
    'overall_accuracy': accuracy,
    'total_images': total_predictions,
    'correct_predictions': correct_predictions,
    'top_15_per_class': top15_per_class,
    'class_statistics': class_stats,
    'all_predictions': all_predictions
}

with open('test_predictions_top15_per_class_seagrass_1shot_filtered.json', 'w') as f:
    json.dump(results_data, f, indent=4, default=str)

print(f"\nResults saved to 'test_predictions_top15_per_class_seagrass_1shot_filtered.json'")

# Optional: Create a summary table
print(f"\n" + "="*80)
print("SUMMARY TABLE - TOP 15 COUNT PER CLASS")
print("="*80)
print(f"{'Class':<15} {'Top 15 Count':<12} {'Avg Confidence':<15} {'Correct in Top 15':<15}")
print("-" * 65)

for class_name, predictions in top15_per_class.items():
    avg_conf = sum(p['confidence'] for p in predictions) / len(predictions)
    correct_count = sum(1 for p in predictions if p['is_correct'])
    print(f"{class_name:<15} {len(predictions):<12} {avg_conf:.4f}{'':>7} {correct_count:<15}")

Using device: cuda

Total loaded: 20872 samples from enhanced results

First sample keys: ['image_path', 'true_label', 'predicted_label', 'is_correct', 'predicted_class_idx', 'confidence', 'entropy', 'distance_to_prototype', 'margin', 'max_confidence', 'avg_dissimilarity', 'min_dissimilarity', 'max_dissimilarity', 'closest_support_image', 'most_dissimilar_support_image', 'dissimilarity_std', 'combined_score']
First sample preview: [('image_path', 'C:/Users/matin/ECU/Test_Patches/Amphibolis/Amphibolis_DBCA_test_100_patch_0.jpg'), ('true_label', 'Amphibolis'), ('predicted_label', 'Amphibolis')]

Processing summary:
  - Total samples processed: 20872
  - Correct predictions: 13543
  - Incorrect predictions: 7329
  - Skipped (no true_label): 0
Training (original): 4
Enhanced data available (correct predictions only):
  Amphibolis: 4993 images (avg conf: 0.994, avg dissim: 0.095)
  Background: 1596 images (avg conf: 0.988, avg dissim: 0.094)
  Halophila: 5460 images (avg conf: 0.996, avg di

Training:   0%|                                                                               | 0/4000 [00:00<?, ?it/s]

Episode    0 | Loss: 1.4142 | Train: 0.317 | Val: 0.580±0.124 | Temp: 10.00 | LR: 1.00e-04


Training:   0%|                                                                    | 1/4000 [00:23<25:33:21, 23.01s/it]

✅ New best: 0.580


...

Training:  99%|█████████████████████████████████████████████████████████████████▏| 3951/4000 [1:57:16<05:23,  6.61s/it]

Episode 3950 | Loss: 0.2030 | Train: 1.000 | Val: 0.587±0.127 | Temp: 10.00 | LR: 1.04e-06


Training: 100%|██████████████████████████████████████████████████████████████████| 4000/4000 [1:58:27<00:00,  1.78s/it]



Training complete! Best validation accuracy: 0.687

Best model loaded for evaluation.

PREDICTING ALL TEST IMAGES AND GETTING TOP 15 BY CONFIDENCE


ValueError: too many values to unpack (expected 2)